<a href="https://colab.research.google.com/github/FuaadBashi/Died-pipeline-COLAB/blob/main/Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers seqeval datasets


import os, json, random
import numpy as np
import torch
from collections import Counter
import torch.nn as nn
from torch.utils.data import Dataset
from collections import Counter
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                          TrainingArguments, Trainer, EarlyStoppingCallback)
from sklearn.utils import resample
from seqeval.metrics import f1_score, classification_report
from datasets import Dataset as HFDataset

!pip install -q torch transformers seqeval datasets

import json, random
import numpy as np
from collections import Counter

from datasets import Dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset as TorchDataset

import random
import numpy as np
import torch
import json
from collections import Counter, defaultdict
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         TrainingArguments, Trainer, EarlyStoppingCallback)
from torch.utils.data import Dataset as TorchDataset
from seqeval.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

import torch.nn.functional as F
# ─── ClinicalBERT Fine-tuning for PHI De-Identification ───────────────────────────────

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.4 MB/s eta 0:00:00


In [3]:
# =========================
# ENHANCED ClinicalBERT - LOCATION & PHONE Optimization
# =========================
import random
import numpy as np
import torch
import json
from collections import Counter, defaultdict
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         TrainingArguments, Trainer, EarlyStoppingCallback)
from torch.utils.data import Dataset as TorchDataset
from seqeval.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# 1) Enhanced Seed & Device Setup
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name()}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3}GB")

# -------------------------
# 2) Load and Validate Data
# -------------------------
def load_and_validate_data(filepath):
    """Load data with validation checks"""
    with open(filepath) as f:
        data = json.load(f)

    valid_data = []
    for i, entry in enumerate(data):
        if len(entry["tokens"]) == len(entry["labels"]):
            valid_data.append(entry)
        else:
            print(f"⚠️ Skipping entry {i}: token-label mismatch")

    return valid_data

train_data = load_and_validate_data('/content/train.json')
val_data = load_and_validate_data('/content/val.json')
test_data = load_and_validate_data('/content/test.json')

train_tokens = [e["tokens"] for e in train_data]
train_labels = [e["labels"] for e in train_data]
val_tokens = [e["tokens"] for e in val_data]
val_labels = [e["labels"] for e in val_data]
test_tokens = [e["tokens"] for e in test_data]

print(f"✅ Data loaded - Train: {len(train_tokens)} | Val: {len(val_tokens)} | Test: {len(test_tokens)}")

# -------------------------
# 3) Enhanced Label Cleaning
# -------------------------
def clean_and_normalize_labels(labels):
    """Clean labels with detailed tracking"""
    out = []
    changes = defaultdict(int)

    for l in labels:
        if l in ["B-AGE", "I-AGE"]:
            out.append("O")
            changes["AGE_removed"] += 1
        elif l in ["B-DOCTOR", "I-DOCTOR"]:
            out.append(l.replace("DOCTOR", "NAME"))
            changes["DOCTOR_to_NAME"] += 1
        elif l in ["B-PATIENT", "I-PATIENT"]:
            out.append(l.replace("PATIENT", "NAME"))
            changes["PATIENT_to_NAME"] += 1
        else:
            out.append(l)
    return out, changes

# Process labels
total_changes = defaultdict(int)
cleaned_train_labels = []
cleaned_val_labels = []

for labels in train_labels:
    clean_labels, changes = clean_and_normalize_labels(labels)
    cleaned_train_labels.append(clean_labels)
    for k, v in changes.items():
        total_changes[k] += v

for labels in val_labels:
    clean_labels, changes = clean_and_normalize_labels(labels)
    cleaned_val_labels.append(clean_labels)

train_labels = cleaned_train_labels
val_labels = cleaned_val_labels

print("📊 Label cleaning statistics:")
for change, count in total_changes.items():
    print(f"   {change}: {count}")

target_labels = [
    "B-DATE", "I-DATE", "B-HOSPITAL", "I-HOSPITAL", "B-ID", "I-ID",
    "B-LOCATION", "I-LOCATION", "B-NAME", "I-NAME", "B-PHONE", "I-PHONE", "O"
]

label_to_id = {l: i for i, l in enumerate(target_labels)}
id_to_label = {i: l for l, i in label_to_id.items()}

# -------------------------
# 4) TARGETED Data Augmentation for LOCATION & PHONE
# -------------------------
def analyze_rare_entities(tokens_list, labels_list):
    """Analyze LOCATION and PHONE entities specifically"""
    entity_stats = defaultdict(lambda: {"count": 0, "samples": [], "contexts": []})

    for sample_idx, (tokens, labels) in enumerate(zip(tokens_list, labels_list)):
        sample_has_location = False
        sample_has_phone = False

        for i, label in enumerate(labels):
            if label != "O":
                entity_type = label[2:] if label.startswith(("B-", "I-")) else label

                if entity_type in ["LOCATION", "PHONE"]:
                    entity_stats[entity_type]["count"] += 1
                    entity_stats[entity_type]["samples"].append(sample_idx)

                    # Capture wider context for rare entities
                    start = max(0, i-5)
                    end = min(len(tokens), i+6)
                    context = " ".join(tokens[start:end])
                    entity_stats[entity_type]["contexts"].append(context)

                    if entity_type == "LOCATION":
                        sample_has_location = True
                    elif entity_type == "PHONE":
                        sample_has_phone = True

        # Track samples containing rare entities
        if sample_has_location:
            entity_stats["LOCATION"]["samples"].append(sample_idx)
        if sample_has_phone:
            entity_stats["PHONE"]["samples"].append(sample_idx)

    return entity_stats

def aggressive_oversample_rare_entities(tokens_list, labels_list, entity_stats):
    """Aggressively oversample LOCATION and PHONE entities"""
    aug_tokens, aug_labels = [], []

    rare_entities = ["LOCATION", "PHONE"]

    for entity in rare_entities:
        if entity not in entity_stats or entity_stats[entity]["count"] == 0:
            print(f"⚠️ No {entity} entities found in training data")
            continue

        unique_samples = list(set(entity_stats[entity]["samples"]))
        current_count = entity_stats[entity]["count"]

        print(f"🎯 Targeting {entity}:")
        print(f"   Current instances: {current_count}")
        print(f"   Samples with {entity}: {len(unique_samples)}")

        if len(unique_samples) == 0:
            continue

        # Extremely aggressive oversampling for rare entities
        target_multiplier = max(20, 100 // len(unique_samples))  # At least 20x, more if very few samples

        print(f"   📈 Oversampling {entity}: {len(unique_samples)} samples × {target_multiplier}")

        # Oversample each unique sample multiple times
        for _ in range(target_multiplier):
            for sample_idx in unique_samples:
                if sample_idx < len(tokens_list):
                    aug_tokens.append(tokens_list[sample_idx])
                    aug_labels.append(labels_list[sample_idx])

    return aug_tokens, aug_labels

# Analyze and oversample rare entities
entity_stats = analyze_rare_entities(train_tokens, train_labels)
print("🔍 Analyzing rare entities...")

for entity in ["LOCATION", "PHONE"]:
    if entity in entity_stats:
        print(f"   {entity}: {entity_stats[entity]['count']} instances")
        if entity_stats[entity]['contexts']:
            print(f"   Sample contexts: {entity_stats[entity]['contexts'][:3]}")

print("\n🚀 Applying aggressive oversampling for LOCATION & PHONE...")
aug_tokens, aug_labels = aggressive_oversample_rare_entities(train_tokens, train_labels, entity_stats)

train_tokens.extend(aug_tokens)
train_labels.extend(aug_labels)

print(f"✅ Aggressive oversampling complete: +{len(aug_tokens)} samples")
print(f"   Total training samples: {len(train_tokens)}")

# -------------------------
# 5) EXTREME Class Weights for Rare Entities
# -------------------------
def calculate_extreme_weights_for_rare(labels_list):
    """Calculate extreme class weights focusing on LOCATION & PHONE"""
    flat_labels = [lbl for seq in labels_list for lbl in seq]
    counts = Counter(flat_labels)
    total = len(flat_labels)

    weights = {}
    for label in target_labels:
        count = counts.get(label, 1)

        if label == "O":
            weights[label] = 0.3  # Moderate O-class reduction
        elif "LOCATION" in label or "PHONE" in label:
            # EXTREME weights for LOCATION and PHONE
            base_weight = total / (len(target_labels) * count)
            weights[label] = min(base_weight * 5.0, 25.0)  # 5x multiplier, cap at 25
            print(f"🎯 Extreme weight for {label}: {weights[label]:.2f}")
        else:
            # Standard weights for other entities
            base_weight = total / (len(target_labels) * count)
            weights[label] = min(base_weight * 1.2, 6.0)

    return weights

# -------------------------
# 6) Enhanced Dataset with Pattern Recognition
# -------------------------
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", use_fast=True)

class EnhancedNERDataset(TorchDataset):
    def __init__(self, tokens, labels, tokenizer, label_to_id, max_len=256):
        self.tokens, self.labels = tokens, labels
        self.tok, self.l2id, self.max_len = tokenizer, label_to_id, max_len

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        words, labs = self.tokens[idx], self.labels[idx]

        # Ensure consistency
        min_len = min(len(words), len(labs))
        words = words[:min_len]
        labs = labs[:min_len]

        # Handle long sequences
        if len(words) > self.max_len - 2:
            words = words[:self.max_len - 2]
            labs = labs[:self.max_len - 2]

        # Enhanced encoding for rare entity detection
        enc = self.tok(words, is_split_into_words=True, truncation=True,
                       max_length=self.max_len, padding="max_length")

        # Align labels with subword tokens
        wids = enc.word_ids()
        aligned = []
        prev_wid = None

        for wid in wids:
            if wid is None:
                aligned.append(-100)
            elif wid != prev_wid:
                if wid < len(labs):
                    label = labs[wid]
                    aligned.append(self.l2id.get(label, self.l2id["O"]))
                else:
                    aligned.append(self.l2id["O"])
            else:
                aligned.append(-100)
            prev_wid = wid

        return {
            "input_ids": torch.tensor(enc["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(enc["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(aligned, dtype=torch.long)
        }

# -------------------------
# 7) Focal Loss Trainer for Rare Entity Focus
# -------------------------
class FocalLossTrainer(Trainer):
    def __init__(self, class_weights=None, focal_gamma=3.0, label_to_id=None, **kwargs):
        super().__init__(**kwargs)
        self.class_weights = class_weights
        self.focal_gamma = focal_gamma  # Higher gamma for harder focus on rare entities
        self.label_to_id = label_to_id or {}

    def focal_loss(self, logits, labels):
        """Enhanced focal loss with extreme focus on rare entities"""
        ce_loss = F.cross_entropy(logits, labels, weight=self.class_weights, reduction='none')
        pt = torch.exp(-ce_loss)

        # Adaptive alpha based on label rarity
        alpha = torch.ones_like(labels, dtype=torch.float)

        # Get rare entity label IDs
        rare_label_ids = [
            self.label_to_id.get("B-LOCATION", -1),
            self.label_to_id.get("I-LOCATION", -1),
            self.label_to_id.get("B-PHONE", -1),
            self.label_to_id.get("I-PHONE", -1)
        ]

        # Apply high alpha for rare entities
        for label_id in rare_label_ids:
            if label_id != -1:  # Valid label ID
                alpha = torch.where(labels == label_id, 0.9, alpha)

        focal_loss = alpha * (1 - pt) ** self.focal_gamma * ce_loss
        return focal_loss.mean()

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Filter active positions
        active_loss = labels.view(-1) != -100
        active_logits = logits.view(-1, logits.size(-1))[active_loss]
        active_labels = labels.view(-1)[active_loss]

        if len(active_labels) == 0:
            return torch.tensor(0.0, requires_grad=True, device=logits.device)

        # Use focal loss for better rare entity detection
        loss = self.focal_loss(active_logits, active_labels)

        return (loss, outputs) if return_outputs else loss

# -------------------------
# 8) Enhanced Metrics Tracking
# -------------------------
def compute_enhanced_metrics(p):
    """Enhanced metrics with specific tracking for LOCATION & PHONE"""
    predictions = np.argmax(p.predictions, axis=-1)
    labels = p.label_ids

    true_labels, pred_labels = [], []

    for pred_seq, label_seq in zip(predictions, labels):
        true_seq, pred_seq_clean = [], []

        for pred, label in zip(pred_seq, label_seq):
            if label != -100:
                true_seq.append(id_to_label[label])
                pred_seq_clean.append(id_to_label[pred])

        if true_seq:
            true_labels.append(true_seq)
            pred_labels.append(pred_seq_clean)

    if not true_labels:
        return {"f1": 0.0, "precision": 0.0, "recall": 0.0}

    # Overall metrics
    f1 = f1_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)

    # Specific metrics for rare entities
    entity_metrics = {}
    for entity in ["LOCATION", "PHONE"]:
        entity_true = []
        entity_pred = []

        for true_seq, pred_seq in zip(true_labels, pred_labels):
            et = [label if entity in label else 'O' for label in true_seq]
            ep = [label if entity in label else 'O' for label in pred_seq]
            entity_true.append(et)
            entity_pred.append(ep)

        try:
            entity_f1 = f1_score(entity_true, entity_pred)
            entity_metrics[f"{entity.lower()}_f1"] = entity_f1
            print(f"📊 {entity} F1 during training: {entity_f1:.4f}")
        except:
            entity_metrics[f"{entity.lower()}_f1"] = 0.0

    return {
        "f1": f1,
        "precision": precision,
        "recall": recall,
        **entity_metrics
    }

# -------------------------
# 9) Enhanced Training Configuration
# -------------------------
def train_enhanced_model():
    """Train with enhanced focus on LOCATION & PHONE"""

    # Calculate extreme weights
    weight_dict = calculate_extreme_weights_for_rare(train_labels)
    class_weights = torch.tensor([weight_dict[label] for label in target_labels],
                               dtype=torch.float).to(device)

    print("\n⚖️ Enhanced class weights (LOCATION & PHONE focus):")
    for label, weight in zip(target_labels, class_weights.cpu().numpy()):
        if "LOCATION" in label or "PHONE" in label:
            print(f"   🎯 {label}: {weight:.2f} (BOOSTED)")
        else:
            print(f"   {label}: {weight:.2f}")

    # Enhanced training arguments
    args = TrainingArguments(
        output_dir="./outputs/clinicalbert_location_phone_enhanced",
        learning_rate=1.5e-5,              # Slightly lower for stability with extreme weights
        per_device_train_batch_size=2,     # Smaller batch for more frequent updates
        gradient_accumulation_steps=8,     # Maintain effective batch size
        per_device_eval_batch_size=4,
        num_train_epochs=12,               # More epochs due to data augmentation
        warmup_ratio=0.15,                 # More warmup for stability
        lr_scheduler_type="cosine_with_restarts",
        weight_decay=0.01,
        max_grad_norm=0.5,                 # Lower grad norm for stability
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        fp16=True,
        dataloader_pin_memory=True,
        dataloader_num_workers=2,
        seed=SEED,
        report_to="none",
        logging_steps=50,
        save_total_limit=5,
        eval_accumulation_steps=4,
        prediction_loss_only=False
    )

    # Initialize fresh model
    model = AutoModelForTokenClassification.from_pretrained(
        "emilyalsentzer/Bio_ClinicalBERT",
        num_labels=len(target_labels),
        id2label=id_to_label,
        label2id=label_to_id,
        hidden_dropout_prob=0.2,          # Slightly higher dropout for regularization
        attention_probs_dropout_prob=0.2
    )

    # Create enhanced datasets
    train_ds = EnhancedNERDataset(train_tokens, train_labels, tokenizer, label_to_id)
    val_ds = EnhancedNERDataset(val_tokens, val_labels, tokenizer, label_to_id)

    # Create focal loss trainer
    trainer = FocalLossTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        processing_class=tokenizer,
        compute_metrics=compute_enhanced_metrics,
        class_weights=class_weights,
        focal_gamma=3.0,  # High gamma for extreme focus
        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
    )

    print("🚀 Starting LOCATION & PHONE enhanced training...")
    print(f"📊 Enhanced Configuration:")
    print(f"   • Learning rate: 1.5e-5 (stable)")
    print(f"   • Epochs: 12 (extended)")
    print(f"   • Batch size: 2 (frequent updates)")
    print(f"   • Focal loss gamma: 3.0 (extreme focus)")
    print(f"   • LOCATION/PHONE samples: {len(aug_tokens)} additional")

    try:
        train_results = trainer.train()
        trainer.save_model("./outputs/clinicalbert_location_phone_enhanced")

        print("✅ Enhanced training completed successfully!")

        # Final evaluation
        eval_results = trainer.evaluate()

        print(f"\n📈 Enhanced Results:")
        print(f"   • Overall F1: {eval_results.get('eval_f1', 0):.4f}")
        print(f"   • Overall Precision: {eval_results.get('eval_precision', 0):.4f}")
        print(f"   • Overall Recall: {eval_results.get('eval_recall', 0):.4f}")
        print(f"   • LOCATION F1: {eval_results.get('eval_location_f1', 0):.4f}")
        print(f"   • PHONE F1: {eval_results.get('eval_phone_f1', 0):.4f}")

        return trainer, eval_results

    except Exception as e:
        print(f"❌ Enhanced training failed: {str(e)}")
        raise

# -------------------------
# 10) Run Enhanced Training
# -------------------------
if __name__ == "__main__":
    trainer, results = train_enhanced_model()

    print(f"\n🎯 LOCATION & PHONE Enhancement Results:")
    print(f"   📊 Expected improvements in rare entity detection")
    print(f"   🚀 Model saved for enhanced PHI detection capabilities")

✅ Using device: cuda
   GPU: Tesla T4
   Memory: 14GB
✅ Data loaded - Train: 669 | Val: 220 | Test: 220
📊 Label cleaning statistics:
   PATIENT_to_NAME: 1796
   DOCTOR_to_NAME: 6363
   AGE_removed: 13
🔍 Analyzing rare entities...
   LOCATION: 360 instances
   Sample contexts: ['SOCIAL HISTORY : Lives in Merca . Drinks ginger brandy to', ': The patient lives in Jer by herself . However ,', 'Lives with her husband in Bayont . PHYSICAL EXAMINATION : She']
   PHONE: 236 instances
   Sample contexts: ["Dr. Stable 's office , 495-3401 . Goal INR should be", 'need additional information please call 605-304-8547 . PCP Name : SC', ', Georgia 97746 . Phone# (139) 667-6656 . Dictated By :']

🚀 Applying aggressive oversampling for LOCATION & PHONE...
🎯 Targeting LOCATION:
   Current instances: 360
   Samples with LOCATION: 90
   📈 Oversampling LOCATION: 90 samples × 20
🎯 Targeting PHONE:
   Current instances: 236
   Samples with PHONE: 148
   📈 Oversampling PHONE: 148 samples × 20
✅ Aggressive ove

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

🎯 Extreme weight for B-LOCATION: 25.00
🎯 Extreme weight for I-LOCATION: 25.00
🎯 Extreme weight for B-PHONE: 25.00
🎯 Extreme weight for I-PHONE: 25.00

⚖️ Enhanced class weights (LOCATION & PHONE focus):
   B-DATE: 6.00
   I-DATE: 6.00
   B-HOSPITAL: 6.00
   I-HOSPITAL: 6.00
   B-ID: 6.00
   I-ID: 6.00
   🎯 B-LOCATION: 25.00 (BOOSTED)
   🎯 I-LOCATION: 25.00 (BOOSTED)
   B-NAME: 6.00
   I-NAME: 6.00
   🎯 B-PHONE: 25.00 (BOOSTED)
   🎯 I-PHONE: 25.00 (BOOSTED)
   O: 0.30


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

🚀 Starting LOCATION & PHONE enhanced training...
📊 Enhanced Configuration:
   • Learning rate: 1.5e-5 (stable)
   • Epochs: 12 (extended)
   • Batch size: 2 (frequent updates)
   • Focal loss gamma: 3.0 (extreme focus)
   • LOCATION/PHONE samples: 4760 additional


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Location F1,Phone F1
1,0.201500,0.164293,0.536398,0.387114,0.873090,0.032587,0.494382
2,0.006200,0.089405,0.824645,0.747137,0.920094,0.178947,0.807018
3,0.004200,0.086143,0.888273,0.840321,0.942029,0.273504,0.807018
4,0.002300,0.077828,0.911190,0.868346,0.958480,0.377358,0.867925
5,0.001100,0.088493,0.908681,0.873239,0.947121,0.404762,0.846154
6,0.000800,0.077320,0.934426,0.910137,0.960047,0.493151,0.938776
7,0.000800,0.083756,0.936300,0.909830,0.964356,0.457831,0.936170
8,0.000400,0.088507,0.926811,0.898199,0.957305,0.450704,0.938776
9,0.000300,0.083578,0.941424,0.920629,0.963181,0.507042,0.936170
10,0.001600,0.081854,0.940681,0.919566,0.962789,0.507042,0.936170


📊 LOCATION F1 during training: 0.0326
📊 PHONE F1 during training: 0.4944
📊 LOCATION F1 during training: 0.1789
📊 PHONE F1 during training: 0.8070
📊 LOCATION F1 during training: 0.2735
📊 PHONE F1 during training: 0.8070
📊 LOCATION F1 during training: 0.3774
📊 PHONE F1 during training: 0.8679
📊 LOCATION F1 during training: 0.4048
📊 PHONE F1 during training: 0.8462
📊 LOCATION F1 during training: 0.4932
📊 PHONE F1 during training: 0.9388
📊 LOCATION F1 during training: 0.4578
📊 PHONE F1 during training: 0.9362
📊 LOCATION F1 during training: 0.4507
📊 PHONE F1 during training: 0.9388
📊 LOCATION F1 during training: 0.5070
📊 PHONE F1 during training: 0.9362
📊 LOCATION F1 during training: 0.5070
📊 PHONE F1 during training: 0.9362
📊 LOCATION F1 during training: 0.4800
📊 PHONE F1 during training: 0.9362
📊 LOCATION F1 during training: 0.4800
📊 PHONE F1 during training: 0.9362
✅ Enhanced training completed successfully!


📊 LOCATION F1 during training: 0.5070
📊 PHONE F1 during training: 0.9362

📈 Enhanced Results:
   • Overall F1: 0.9414
   • Overall Precision: 0.9206
   • Overall Recall: 0.9632
   • LOCATION F1: 0.5070
   • PHONE F1: 0.9362

🎯 LOCATION & PHONE Enhancement Results:
   📊 Expected improvements in rare entity detection
   🚀 Model saved for enhanced PHI detection capabilities


In [2]:
# =========================
# FIXED ClinicalBERT Evaluation Suite
# =========================
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter, defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# 1) Safe Evaluation Functions (Handle Length Mismatches)
# -------------------------
def safe_sequence_metrics(true_labels, pred_labels):
    """Calculate metrics safely handling sequence length mismatches"""

    # Ensure sequences have same length
    aligned_true = []
    aligned_pred = []

    for true_seq, pred_seq in zip(true_labels, pred_labels):
        min_len = min(len(true_seq), len(pred_seq))
        if min_len > 0:
            aligned_true.append(true_seq[:min_len])
            aligned_pred.append(pred_seq[:min_len])

    if not aligned_true:
        return {"f1": 0.0, "precision": 0.0, "recall": 0.0}

    # Calculate token-level metrics
    all_true_flat = [label for seq in aligned_true for label in seq]
    all_pred_flat = [label for seq in aligned_pred for label in seq]

    # Entity-level metrics (safer approach)
    true_entities = set()
    pred_entities = set()

    for seq_idx, (true_seq, pred_seq) in enumerate(zip(aligned_true, aligned_pred)):
        # Extract entities from true sequence
        current_entity = None
        entity_start = None

        for pos, label in enumerate(true_seq):
            if label.startswith('B-'):
                if current_entity is not None:
                    true_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = label[2:]
                entity_start = pos
            elif label.startswith('I-') and current_entity is not None:
                continue  # Continue current entity
            else:  # 'O' or different entity
                if current_entity is not None:
                    true_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = None
                entity_start = None

        # Handle entity at end of sequence
        if current_entity is not None:
            true_entities.add((seq_idx, entity_start, len(true_seq)-1, current_entity))

        # Extract entities from predicted sequence
        current_entity = None
        entity_start = None

        for pos, label in enumerate(pred_seq):
            if label.startswith('B-'):
                if current_entity is not None:
                    pred_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = label[2:]
                entity_start = pos
            elif label.startswith('I-') and current_entity is not None:
                continue
            else:
                if current_entity is not None:
                    pred_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = None
                entity_start = None

        if current_entity is not None:
            pred_entities.add((seq_idx, entity_start, len(pred_seq)-1, current_entity))

    # Calculate metrics
    tp = len(true_entities & pred_entities)
    fp = len(pred_entities - true_entities)
    fn = len(true_entities - pred_entities)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "true_entities": len(true_entities),
        "pred_entities": len(pred_entities),
        "tp": tp,
        "fp": fp,
        "fn": fn
    }

# -------------------------
# 2) Fixed Prediction Function
# -------------------------
def predict_entities_fixed(tokens_list, model, tokenizer, max_len=256):
    """Generate predictions with better alignment"""
    predictions = []

    model.eval()
    with torch.no_grad():
        for tokens in tokens_list:
            try:
                # Limit sequence length
                if len(tokens) > max_len - 2:
                    tokens = tokens[:max_len - 2]

                # Tokenize
                encoding = tokenizer(
                    tokens,
                    is_split_into_words=True,
                    truncation=True,
                    max_length=max_len,
                    padding="max_length",
                    return_tensors="pt"
                )

                # Get word IDs for alignment
                word_ids = encoding.word_ids()

                # Move to device
                device = next(model.parameters()).device
                encoding = {k: v.to(device) for k, v in encoding.items()}

                # Predict
                outputs = model(**encoding)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1).cpu().numpy()[0]

                # Align predictions with original tokens
                aligned_preds = []
                prev_word_id = None

                for i, word_id in enumerate(word_ids):
                    if word_id is None:
                        continue  # Skip special tokens
                    elif word_id != prev_word_id:
                        # First subword of a word
                        if word_id < len(tokens):
                            pred_id = preds[i]
                            pred_label = id_to_label.get(pred_id, "O")
                            aligned_preds.append(pred_label)
                    prev_word_id = word_id

                # Ensure same length as input tokens
                while len(aligned_preds) < len(tokens):
                    aligned_preds.append("O")
                aligned_preds = aligned_preds[:len(tokens)]

                predictions.append(aligned_preds)

            except Exception as e:
                print(f"Error processing sequence: {e}")
                # Fallback: all O labels
                predictions.append(["O"] * len(tokens))

    return predictions

# -------------------------
# 3) Load Model and Data (Fixed Path)
# -------------------------
print("🔄 Loading trained model and tokenizer...")

# Try multiple possible model paths - UPDATED for enhanced model
possible_paths = [
    "./outputs/clinicalbert_location_phone_enhanced",  # NEW enhanced model
    "./outputs/clinicalbert_optimized",                # Previous optimized model
    "./outputs/clinicalbert_max_recall",               # Original model
    "/content/outputs/clinicalbert_location_phone_enhanced",
    "/content/outputs/clinicalbert_optimized",
    "/content/outputs/clinicalbert_max_recall"
]

model = None
model_path = None

try:
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", use_fast=True)
    print("✅ Tokenizer loaded")

    # Try to find the model
    for path in possible_paths:
        try:
            print(f"🔍 Trying to load model from: {path}")
            model = AutoModelForTokenClassification.from_pretrained(path)
            model_path = path
            print(f"✅ Model loaded from: {path}")
            break
        except Exception as e:
            print(f"❌ Failed to load from {path}: {str(e)[:50]}...")
            continue

    if model is None:
        print("❌ Could not find trained model. Please check if training completed successfully.")
        print("💡 Available files in current directory:")
        import os
        if os.path.exists("outputs"):
            print(f"   outputs/ contents: {os.listdir('outputs')}")
        else:
            print("   No outputs/ directory found")
        exit()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print(f"✅ Model ready on {device}")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

# Load data
try:
    with open('/content/test.json') as f:
        test_data = json.load(f)
    test_tokens = [entry["tokens"] for entry in test_data]

    with open('/content/val.json') as f:
        gt_data = json.load(f)
    gt_tokens = [entry["tokens"] for entry in gt_data]
    gt_labels = [entry["labels"] for entry in gt_data]

    print(f"✅ Data loaded: Test={len(test_tokens)}, GT={len(gt_tokens)} samples")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    exit()

# Clean labels (same as training)
def clean_and_normalize_labels(labels):
    out = []
    for l in labels:
        if l in ["B-AGE", "I-AGE"]:
            out.append("O")
        elif l in ["B-DOCTOR", "I-DOCTOR"]:
            out.append(l.replace("DOCTOR", "NAME"))
        elif l in ["B-PATIENT", "I-PATIENT"]:
            out.append(l.replace("PATIENT", "NAME"))
        else:
            out.append(l)
    return out

gt_labels_clean = [clean_and_normalize_labels(labels) for labels in gt_labels]

# Label mappings
target_labels = [
    "B-DATE", "I-DATE", "B-HOSPITAL", "I-HOSPITAL", "B-ID", "I-ID",
    "B-LOCATION", "I-LOCATION", "B-NAME", "I-NAME", "B-PHONE", "I-PHONE", "O"
]
label_to_id = {l: i for i, l in enumerate(target_labels)}
id_to_label = {i: l for l, i in label_to_id.items()}

# -------------------------
# 4) Generate Predictions with Fixed Function
# -------------------------
print("🔄 Generating predictions...")
gt_predictions = predict_entities_fixed(gt_tokens, model, tokenizer)
print(f"✅ Generated predictions for {len(gt_predictions)} samples")

# -------------------------
# 5) Safe Evaluation
# -------------------------
print("🔄 Evaluating predictions...")
evaluation_results = safe_sequence_metrics(gt_labels_clean, gt_predictions)

# Per-entity evaluation
entity_metrics = {}
entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]

for entity in entities:
    # Filter sequences for this entity
    entity_true = []
    entity_pred = []

    for true_seq, pred_seq in zip(gt_labels_clean, gt_predictions):
        et = [label if entity in label else 'O' for label in true_seq]
        ep = [label if entity in label else 'O' for label in pred_seq]
        entity_true.append(et)
        entity_pred.append(ep)

    entity_result = safe_sequence_metrics(entity_true, entity_pred)
    entity_metrics[entity] = {
        'f1': entity_result['f1'],
        'precision': entity_result['precision'],
        'recall': entity_result['recall']
    }

# -------------------------
# 6) Save Results
# -------------------------
results_data = {
    'model_path': model_path,
    'test_samples': len(gt_tokens),
    'overall_metrics': {
        'f1': evaluation_results['f1'],
        'precision': evaluation_results['precision'],
        'recall': evaluation_results['recall']
    },
    'entity_metrics': entity_metrics,
    'detailed_stats': {
        'true_entities': evaluation_results['true_entities'],
        'pred_entities': evaluation_results['pred_entities'],
        'tp': evaluation_results['tp'],
        'fp': evaluation_results['fp'],
        'fn': evaluation_results['fn']
    },
    'predictions': gt_predictions,
    'true_labels': gt_labels_clean,
    'tokens': gt_tokens
}

# Save results
with open('evaluation_results_clincalbert.json', 'w') as f:
    json.dump(results_data, f, indent=2)

# Create entity DataFrame
entity_df = pd.DataFrame(entity_metrics).T
entity_df.to_csv('entity_performance_clincalbert.csv')

print("✅ Results saved successfully!")

# -------------------------
# 7) Print Summary
# -------------------------
print("\n" + "="*60)
print("🎯 EVALUATION RESULTS (FIXED)")
print("="*60)

print(f"\n📊 Overall Performance:")
print(f"   • F1-Score:  {evaluation_results['f1']:.4f}")
print(f"   • Precision: {evaluation_results['precision']:.4f}")
print(f"   • Recall:    {evaluation_results['recall']:.4f}")

print(f"\n📈 Entity Counts:")
print(f"   • True entities found: {evaluation_results['true_entities']}")
print(f"   • Predicted entities: {evaluation_results['pred_entities']}")
print(f"   • True Positives: {evaluation_results['tp']}")
print(f"   • False Positives: {evaluation_results['fp']}")
print(f"   • False Negatives: {evaluation_results['fn']}")

print(f"\n🎯 Entity-Level Performance:")
for entity, metrics in entity_metrics.items():
    print(f"   • {entity:10s}: F1={metrics['f1']:.3f}, P={metrics['precision']:.3f}, R={metrics['recall']:.3f}")

print(f"\n✅ Evaluation completed successfully!")
print(f"📁 Files saved:")
print(f"   • evaluation_results.json")
print(f"   • entity_performance.csv")

🔄 Loading trained model and tokenizer...
✅ Tokenizer loaded
🔍 Trying to load model from: ./outputs/clinicalbert_location_phone_enhanced
✅ Model loaded from: ./outputs/clinicalbert_location_phone_enhanced
✅ Model ready on cuda
✅ Data loaded: Test=220, GT=220 samples
🔄 Generating predictions...
✅ Generated predictions for 220 samples
🔄 Evaluating predictions...
✅ Results saved successfully!

🎯 EVALUATION RESULTS (FIXED)

📊 Overall Performance:
   • F1-Score:  0.8852
   • Precision: 0.9271
   • Recall:    0.8468

📈 Entity Counts:
   • True entities found: 2899
   • Predicted entities: 2648
   • True Positives: 2455
   • False Positives: 193
   • False Negatives: 444

🎯 Entity-Level Performance:
   • DATE      : F1=0.894, P=0.962, R=0.835
   • HOSPITAL  : F1=0.780, P=0.804, R=0.758
   • ID        : F1=0.984, P=0.996, R=0.973
   • LOCATION  : F1=0.450, P=0.419, R=0.486
   • NAME      : F1=0.761, P=0.845, R=0.692
   • PHONE     : F1=0.880, P=0.957, R=0.815

✅ Evaluation completed successfull

In [ ]:
# ===========================
# FIXED MODEL DOWNLOAD CODE
# ===========================

import os
from google.colab import files

print("📦 Preparing ClinicalBERT model for download...")

# Check what files exist in the outputs directory
if os.path.exists('./outputs'):
    print("✅ Found outputs directory")

    # List contents of outputs directory
    print("\n📂 Contents of ./outputs/:")
    for root, dirs, files_list in os.walk('./outputs'):
        level = root.replace('./outputs', '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files_list:
            file_size = os.path.getsize(os.path.join(root, file))
            file_size_mb = file_size / (1024 * 1024)
            print(f"{subindent}{file} ({file_size_mb:.1f} MB)")
else:
    print("❌ outputs directory not found")

    # Check for alternative locations
    print("\n🔍 Checking alternative model save locations...")

    # Check current directory
    current_files = [f for f in os.listdir('.') if 'clinicalbert' in f.lower() or 'model' in f.lower()]
    if current_files:
        print(f"✅ Found model files in current directory: {current_files}")

    # Check if trainer saved model elsewhere
    if 'trainer' in globals():
        print(f"✅ Trainer output directory: {trainer.args.output_dir}")
        if os.path.exists(trainer.args.output_dir):
            print("✅ Trainer output directory exists")
        else:
            print("❌ Trainer output directory not found")

# Method 1: Try original path
if os.path.exists('./outputs/clinicalbert'):
    print("\n📦 Creating zip file from ./outputs/clinicalbert...")
    !zip -rq clinicalbert_model.zip ./outputs/clinicalbert

    if os.path.exists('clinicalbert_model.zip'):
        print("✅ Zip file created successfully")

        # Check zip file size
        zip_size = os.path.getsize('clinicalbert_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"📦 Zip file size: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting download...")
        files.download('clinicalbert_model.zip')
        print("✅ Download initiated!")
    else:
        print("❌ Failed to create zip file")

# Method 2: Try trainer output directory
elif 'trainer' in globals() and os.path.exists(trainer.args.output_dir):
    print(f"\n📦 Creating zip file from trainer output: {trainer.args.output_dir}")
    !zip -rq clinicalbert_model.zip {trainer.args.output_dir}

    if os.path.exists('clinicalbert_model.zip'):
        print("✅ Zip file created successfully")

        # Check zip file size
        zip_size = os.path.getsize('clinicalbert_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"📦 Zip file size: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting download...")
        files.download('clinicalbert_model.zip')
        print("✅ Download initiated!")
    else:
        print("❌ Failed to create zip file")

# Method 3: Manual save if trainer exists
elif 'trainer' in globals():
    print("\n💾 Manually saving model files...")

    # Create a directory for the model
    os.makedirs('clinicalbert_manual', exist_ok=True)

    # Save the model and tokenizer
    trainer.save_model('clinicalbert_manual')

    if 'tokenizer' in globals():
        tokenizer.save_pretrained('clinicalbert_manual')
        print("✅ Tokenizer saved")

    # Save additional files
    import json

    # Save label mappings
    if 'id_to_label' in globals():
        with open('clinicalbert_manual/id_to_label.json', 'w') as f:
            json.dump(id_to_label, f, indent=2)
        print("✅ Label mappings saved")

    if 'label_to_id' in globals():
        with open('clinicalbert_manual/label_to_id.json', 'w') as f:
            json.dump(label_to_id, f, indent=2)

    # Save training args
    if hasattr(trainer, 'args'):
        trainer.args.save_to_json('clinicalbert_manual/training_args.json')
        print("✅ Training arguments saved")

    # Create zip file
    print("📦 Creating zip file...")
    !zip -rq clinicalbert_model.zip clinicalbert_manual/

    if os.path.exists('clinicalbert_model.zip'):
        zip_size = os.path.getsize('clinicalbert_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"✅ Zip file created: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting download...")
        files.download('clinicalbert_model.zip')
        print("✅ Download initiated!")
    else:
        print("❌ Failed to create zip file")

else:
    print("❌ No model found to download")
    print("\n🔧 Troubleshooting steps:")
    print("1. Make sure your model training completed successfully")
    print("2. Check if trainer.save_model() was called")
    print("3. Verify the output directory path")
    print("4. Try running: trainer.save_model('./clinicalbert_download')")

print("\n📋 What's included in the model zip:")
print("✅ Model weights (pytorch_model.bin)")
print("✅ Model configuration (config.json)")
print("✅ Tokenizer files")
print("✅ Label mappings (id_to_label.json)")
print("✅ Training arguments")
print("\n🚀 You can load this model later with:")
print("   from transformers import AutoModelForTokenClassification, AutoTokenizer")
print("   model = AutoModelForTokenClassification.from_pretrained('./path_to_unzipped_model')")
print("   tokenizer = AutoTokenizer.from_pretrained('./path_to_unzipped_model')")

In [3]:
# =========================
# BIOBERT TRAINING FOR PHI DETECTION
# =========================
import random
import numpy as np
import torch
import json
from collections import Counter, defaultdict
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         TrainingArguments, Trainer, EarlyStoppingCallback)
from torch.utils.data import Dataset as TorchDataset
from seqeval.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

print("🧬 Starting BioBERT Training for PHI Detection")
print("="*60)

# -------------------------
# 1) Setup and Data Loading (Same as previous)
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# Load and clean data (reuse previous functions)
def load_and_validate_data(filepath):
    with open(filepath) as f:
        data = json.load(f)
    return [entry for entry in data if len(entry["tokens"]) == len(entry["labels"])]

def clean_and_normalize_labels(labels):
    out = []
    for l in labels:
        if l in ["B-AGE", "I-AGE"]:
            out.append("O")
        elif l in ["B-DOCTOR", "I-DOCTOR"]:
            out.append(l.replace("DOCTOR", "NAME"))
        elif l in ["B-PATIENT", "I-PATIENT"]:
            out.append(l.replace("PATIENT", "NAME"))
        else:
            out.append(l)
    return out

# Load data
train_data = load_and_validate_data('/content/train.json')
val_data = load_and_validate_data('/content/val.json')
test_data = load_and_validate_data('/content/test.json')

train_tokens = [e["tokens"] for e in train_data]
train_labels = [clean_and_normalize_labels(e["labels"]) for e in train_data]
val_tokens = [e["tokens"] for e in val_data]
val_labels = [clean_and_normalize_labels(e["labels"]) for e in val_data]

print(f"✅ Data loaded - Train: {len(train_tokens)} | Val: {len(val_tokens)}")

# Label setup
target_labels = [
    "B-DATE", "I-DATE", "B-HOSPITAL", "I-HOSPITAL", "B-ID", "I-ID",
    "B-LOCATION", "I-LOCATION", "B-NAME", "I-NAME", "B-PHONE", "I-PHONE", "O"
]
label_to_id = {l: i for i, l in enumerate(target_labels)}
id_to_label = {i: l for l, i in label_to_id.items()}

# -------------------------
# 2) BioBERT-Specific Enhancements
# -------------------------
def analyze_biobert_suitability(tokens_list, labels_list):
    """Analyze dataset for BioBERT-specific characteristics"""
    print("🔬 Analyzing dataset for BioBERT training...")

    medical_terms = ['patient', 'hospital', 'doctor', 'medical', 'clinic', 'physician',
                    'diagnosis', 'treatment', 'medicine', 'therapy', 'surgery']

    medical_term_count = 0
    total_tokens = 0

    for tokens in tokens_list:
        for token in tokens:
            total_tokens += 1
            if any(term in token.lower() for term in medical_terms):
                medical_term_count += 1

    medical_density = medical_term_count / total_tokens * 100
    print(f"   📊 Medical term density: {medical_density:.2f}%")
    print(f"   🎯 BioBERT advantage: Biomedical vocabulary pre-training")

    return medical_density

medical_density = analyze_biobert_suitability(train_tokens, train_labels)

# -------------------------
# 3) BioBERT Dataset Class
# -------------------------
class BioBERTNERDataset(TorchDataset):
    def __init__(self, tokens, labels, tokenizer, label_to_id, max_len=256):
        self.tokens, self.labels = tokens, labels
        self.tok, self.l2id, self.max_len = tokenizer, label_to_id, max_len

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        words, labs = self.tokens[idx], self.labels[idx]

        # Ensure consistency
        min_len = min(len(words), len(labs))
        words = words[:min_len]
        labs = labs[:min_len]

        # Handle long sequences
        if len(words) > self.max_len - 2:
            words = words[:self.max_len - 2]
            labs = labs[:self.max_len - 2]

        # BioBERT tokenization (similar to BERT but with biomedical vocabulary)
        enc = self.tok(words, is_split_into_words=True, truncation=True,
                       max_length=self.max_len, padding="max_length")

        # Align labels with subword tokens
        wids = enc.word_ids()
        aligned = []
        prev_wid = None

        for wid in wids:
            if wid is None:
                aligned.append(-100)
            elif wid != prev_wid:
                if wid < len(labs):
                    label = labs[wid]
                    aligned.append(self.l2id.get(label, self.l2id["O"]))
                else:
                    aligned.append(self.l2id["O"])
            else:
                aligned.append(-100)
            prev_wid = wid

        return {
            "input_ids": torch.tensor(enc["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(enc["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(aligned, dtype=torch.long)
        }

# -------------------------
# 4) BioBERT Training Configuration
# -------------------------
def calculate_balanced_weights(labels_list):
    """Calculate balanced class weights for BioBERT"""
    flat_labels = [lbl for seq in labels_list for lbl in seq]
    counts = Counter(flat_labels)
    total = len(flat_labels)

    weights = {}
    for label in target_labels:
        count = counts.get(label, 1)
        if label == "O":
            weights[label] = 0.5
        else:
            base_weight = total / (len(target_labels) * count)
            weights[label] = min(base_weight * 1.5, 8.0)

    return weights

# -------------------------
# 5) BioBERT Metrics
# -------------------------
def compute_biobert_metrics(p):
    """Compute comprehensive metrics for BioBERT"""
    predictions = np.argmax(p.predictions, axis=-1)
    labels = p.label_ids

    true_labels, pred_labels = [], []

    for pred_seq, label_seq in zip(predictions, labels):
        true_seq, pred_seq_clean = [], []
        for pred, label in zip(pred_seq, label_seq):
            if label != -100:
                true_seq.append(id_to_label[label])
                pred_seq_clean.append(id_to_label[pred])
        if true_seq:
            true_labels.append(true_seq)
            pred_labels.append(pred_seq_clean)

    if not true_labels:
        return {"f1": 0.0, "precision": 0.0, "recall": 0.0}

    f1 = f1_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)

    return {"f1": f1, "precision": precision, "recall": recall}

# -------------------------
# 6) BioBERT Training
# -------------------------
def train_biobert_model():
    """Train BioBERT for PHI detection"""

    print("\n🧬 Initializing BioBERT...")

    # Load BioBERT tokenizer and model
    model_name = "dmis-lab/biobert-base-cased-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    print(f"✅ BioBERT tokenizer loaded")
    print(f"   Vocab size: {tokenizer.vocab_size}")
    print(f"   Model: {model_name}")

    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=len(target_labels),
        id2label=id_to_label,
        label2id=label_to_id,
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1
    )

    # Calculate class weights
    weight_dict = calculate_balanced_weights(train_labels)
    class_weights = torch.tensor([weight_dict[label] for label in target_labels],
                               dtype=torch.float).to(device)

    print("\n⚖️ BioBERT class weights:")
    for label, weight in zip(target_labels, class_weights.cpu().numpy()):
        if "LOCATION" in label or "PHONE" in label:
            print(f"   🎯 {label}: {weight:.2f}")
        else:
            print(f"   {label}: {weight:.2f}")

    # Create datasets
    train_ds = BioBERTNERDataset(train_tokens, train_labels, tokenizer, label_to_id)
    val_ds = BioBERTNERDataset(val_tokens, val_labels, tokenizer, label_to_id)

    # Training arguments optimized for BioBERT
    args = TrainingArguments(
        output_dir="./outputs/biobert_phi_detection",
        learning_rate=2e-5,                # Standard BERT learning rate
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        num_train_epochs=8,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine_with_restarts",
        weight_decay=0.01,
        max_grad_norm=1.0,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        fp16=True,
        dataloader_pin_memory=True,
        dataloader_num_workers=2,
        seed=SEED,
        report_to="none",
        logging_steps=50,
        save_total_limit=3,
        eval_accumulation_steps=4,
        prediction_loss_only=False
    )

    # Custom trainer with class weights
    class BioBERTTrainer(Trainer):
        def __init__(self, class_weights=None, **kwargs):
            super().__init__(**kwargs)
            self.class_weights = class_weights

        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.pop("labels")
            outputs = model(**inputs)
            logits = outputs.logits

            # Filter active positions
            active_loss = labels.view(-1) != -100
            active_logits = logits.view(-1, logits.size(-1))[active_loss]
            active_labels = labels.view(-1)[active_loss]

            if len(active_labels) == 0:
                return torch.tensor(0.0, requires_grad=True, device=logits.device)

            # Weighted cross-entropy loss
            loss = F.cross_entropy(active_logits, active_labels, weight=self.class_weights)
            return (loss, outputs) if return_outputs else loss

    # Create trainer
    trainer = BioBERTTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        processing_class=tokenizer,
        compute_metrics=compute_biobert_metrics,
        class_weights=class_weights,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    print("\n🚀 Starting BioBERT training...")
    print(f"📊 Configuration:")
    print(f"   • Model: BioBERT (biomedical pre-trained)")
    print(f"   • Learning rate: 2e-5")
    print(f"   • Epochs: 8")
    print(f"   • Batch size: 4")
    print(f"   • Medical term density: {medical_density:.2f}%")

    try:
        train_results = trainer.train()
        trainer.save_model("./outputs/biobert_phi_detection")

        print("✅ BioBERT training completed successfully!")

        # Final evaluation
        eval_results = trainer.evaluate()

        print(f"\n📈 BioBERT Results:")
        print(f"   • F1-Score: {eval_results.get('eval_f1', 0):.4f}")
        print(f"   • Precision: {eval_results.get('eval_precision', 0):.4f}")
        print(f"   • Recall: {eval_results.get('eval_recall', 0):.4f}")

        # Save results for comparison
        biobert_results = {
            'model': 'BioBERT',
            'model_name': model_name,
            'f1': eval_results.get('eval_f1', 0),
            'precision': eval_results.get('eval_precision', 0),
            'recall': eval_results.get('eval_recall', 0),
            'medical_density': medical_density
        }

        with open('biobert_results.json', 'w') as f:
            json.dump(biobert_results, f, indent=2)

        print(f"\n🧬 BioBERT advantages observed:")
        print(f"   • Biomedical vocabulary alignment")
        print(f"   • PubMed/PMC pre-training benefits")
        print(f"   • Medical terminology understanding")

        return trainer, eval_results

    except Exception as e:
        print(f"❌ BioBERT training failed: {str(e)}")
        raise

# Run BioBERT training
trainer, results = train_biobert_model()

print(f"\n🎯 BioBERT Training Complete!")
print(f"📁 Model saved to: ./outputs/biobert_phi_detection")
print(f"📊 Results saved to: biobert_results.json")

🧬 Starting BioBERT Training for PHI Detection
✅ Using device: cuda
✅ Data loaded - Train: 669 | Val: 220
🔬 Analyzing dataset for BioBERT training...
   📊 Medical term density: 2.22%
   🎯 BioBERT advantage: Biomedical vocabulary pre-training

🧬 Initializing BioBERT...


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

✅ BioBERT tokenizer loaded
   Vocab size: 28996
   Model: dmis-lab/biobert-base-cased-v1.1


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

...


⚖️ BioBERT class weights:
   B-DATE: 8.00
   I-DATE: 8.00
   B-HOSPITAL: 8.00
   I-HOSPITAL: 8.00
   B-ID: 8.00
   I-ID: 8.00
   🎯 B-LOCATION: 8.00
   🎯 I-LOCATION: 8.00
   B-NAME: 8.00
   I-NAME: 8.00
   🎯 B-PHONE: 8.00
   🎯 I-PHONE: 8.00
   O: 0.50

🚀 Starting BioBERT training...
📊 Configuration:
   • Model: BioBERT (biomedical pre-trained)
   • Learning rate: 2e-5
   • Epochs: 8
   • Batch size: 4
   • Medical term density: 2.22%


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,No log,0.446588,0.777634,0.737360,0.822562
2,1.443100,0.124913,0.917647,0.889952,0.947121
3,0.129100,0.086421,0.948989,0.933005,0.965531
4,0.040500,0.068307,0.952253,0.936388,0.968664
5,0.021700,0.068188,0.963538,0.954284,0.972973
6,0.014500,0.061473,0.962260,0.951033,0.973756
7,0.014500,0.059301,0.962089,0.950325,0.974148
8,0.012500,0.059626,0.961501,0.949924,0.973365


✅ BioBERT training completed successfully!



📈 BioBERT Results:
   • F1-Score: 0.9635
   • Precision: 0.9543
   • Recall: 0.9730

🧬 BioBERT advantages observed:
   • Biomedical vocabulary alignment
   • PubMed/PMC pre-training benefits
   • Medical terminology understanding

🎯 BioBERT Training Complete!
📁 Model saved to: ./outputs/biobert_phi_detection
📊 Results saved to: biobert_results.json


In [4]:
# =========================
# FIXED BioBERT Evaluation Suite
# =========================
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter, defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# 1) Safe Evaluation Functions (Handle Length Mismatches)
# -------------------------
def safe_sequence_metrics(true_labels, pred_labels):
    """Calculate metrics safely handling sequence length mismatches"""

    # Ensure sequences have same length
    aligned_true = []
    aligned_pred = []

    for true_seq, pred_seq in zip(true_labels, pred_labels):
        min_len = min(len(true_seq), len(pred_seq))
        if min_len > 0:
            aligned_true.append(true_seq[:min_len])
            aligned_pred.append(pred_seq[:min_len])

    if not aligned_true:
        return {"f1": 0.0, "precision": 0.0, "recall": 0.0}

    # Calculate token-level metrics
    all_true_flat = [label for seq in aligned_true for label in seq]
    all_pred_flat = [label for seq in aligned_pred for label in seq]

    # Entity-level metrics (safer approach)
    true_entities = set()
    pred_entities = set()

    for seq_idx, (true_seq, pred_seq) in enumerate(zip(aligned_true, aligned_pred)):
        # Extract entities from true sequence
        current_entity = None
        entity_start = None

        for pos, label in enumerate(true_seq):
            if label.startswith('B-'):
                if current_entity is not None:
                    true_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = label[2:]
                entity_start = pos
            elif label.startswith('I-') and current_entity is not None:
                continue  # Continue current entity
            else:  # 'O' or different entity
                if current_entity is not None:
                    true_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = None
                entity_start = None

        # Handle entity at end of sequence
        if current_entity is not None:
            true_entities.add((seq_idx, entity_start, len(true_seq)-1, current_entity))

        # Extract entities from predicted sequence
        current_entity = None
        entity_start = None

        for pos, label in enumerate(pred_seq):
            if label.startswith('B-'):
                if current_entity is not None:
                    pred_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = label[2:]
                entity_start = pos
            elif label.startswith('I-') and current_entity is not None:
                continue
            else:
                if current_entity is not None:
                    pred_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = None
                entity_start = None

        if current_entity is not None:
            pred_entities.add((seq_idx, entity_start, len(pred_seq)-1, current_entity))

    # Calculate metrics
    tp = len(true_entities & pred_entities)
    fp = len(pred_entities - true_entities)
    fn = len(true_entities - pred_entities)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "true_entities": len(true_entities),
        "pred_entities": len(pred_entities),
        "tp": tp,
        "fp": fp,
        "fn": fn
    }

# -------------------------
# 2) Fixed Prediction Function
# -------------------------
def predict_entities_fixed(tokens_list, model, tokenizer, max_len=256):
    """Generate predictions with better alignment"""
    predictions = []

    model.eval()
    with torch.no_grad():
        for tokens in tokens_list:
            try:
                # Limit sequence length
                if len(tokens) > max_len - 2:
                    tokens = tokens[:max_len - 2]

                # Tokenize
                encoding = tokenizer(
                    tokens,
                    is_split_into_words=True,
                    truncation=True,
                    max_length=max_len,
                    padding="max_length",
                    return_tensors="pt"
                )

                # Get word IDs for alignment
                word_ids = encoding.word_ids()

                # Move to device
                device = next(model.parameters()).device
                encoding = {k: v.to(device) for k, v in encoding.items()}

                # Predict
                outputs = model(**encoding)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1).cpu().numpy()[0]

                # Align predictions with original tokens
                aligned_preds = []
                prev_word_id = None

                for i, word_id in enumerate(word_ids):
                    if word_id is None:
                        continue  # Skip special tokens
                    elif word_id != prev_word_id:
                        # First subword of a word
                        if word_id < len(tokens):
                            pred_id = preds[i]
                            pred_label = id_to_label.get(pred_id, "O")
                            aligned_preds.append(pred_label)
                    prev_word_id = word_id

                # Ensure same length as input tokens
                while len(aligned_preds) < len(tokens):
                    aligned_preds.append("O")
                aligned_preds = aligned_preds[:len(tokens)]

                predictions.append(aligned_preds)

            except Exception as e:
                print(f"Error processing sequence: {e}")
                # Fallback: all O labels
                predictions.append(["O"] * len(tokens))

    return predictions

# -------------------------
# 3) Load Model and Data
# -------------------------
print("🧬 Loading BioBERT trained model and tokenizer...")

# Try multiple possible BioBERT model paths
possible_paths = [
    "./outputs/biobert_phi_detection",
    "/content/outputs/biobert_phi_detection",
    "./biobert_phi_detection",
    "/content/biobert_phi_detection"
]

model = None
model_path = None

try:
    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1", use_fast=True)
    print("✅ BioBERT tokenizer loaded")

    # Try to find the model
    for path in possible_paths:
        try:
            print(f"🔍 Trying to load BioBERT model from: {path}")
            model = AutoModelForTokenClassification.from_pretrained(path)
            model_path = path
            print(f"✅ BioBERT model loaded from: {path}")
            break
        except Exception as e:
            print(f"❌ Failed to load from {path}: {str(e)[:50]}...")
            continue

    if model is None:
        print("❌ Could not find trained BioBERT model. Please check if training completed successfully.")
        print("💡 Available files in current directory:")
        import os
        if os.path.exists("outputs"):
            print(f"   outputs/ contents: {os.listdir('outputs')}")
        else:
            print("   No outputs/ directory found")
        exit()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print(f"✅ BioBERT model ready on {device}")

except Exception as e:
    print(f"❌ Error loading BioBERT model: {e}")
    exit()

# Load data
try:
    with open('/content/test.json') as f:
        test_data = json.load(f)
    test_tokens = [entry["tokens"] for entry in test_data]

    with open('/content/val.json') as f:
        gt_data = json.load(f)
    gt_tokens = [entry["tokens"] for entry in gt_data]
    gt_labels = [entry["labels"] for entry in gt_data]

    print(f"✅ Data loaded: Test={len(test_tokens)}, GT={len(gt_tokens)} samples")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    exit()

# Clean labels (same as training)
def clean_and_normalize_labels(labels):
    out = []
    for l in labels:
        if l in ["B-AGE", "I-AGE"]:
            out.append("O")
        elif l in ["B-DOCTOR", "I-DOCTOR"]:
            out.append(l.replace("DOCTOR", "NAME"))
        elif l in ["B-PATIENT", "I-PATIENT"]:
            out.append(l.replace("PATIENT", "NAME"))
        else:
            out.append(l)
    return out

gt_labels_clean = [clean_and_normalize_labels(labels) for labels in gt_labels]

# Label mappings
target_labels = [
    "B-DATE", "I-DATE", "B-HOSPITAL", "I-HOSPITAL", "B-ID", "I-ID",
    "B-LOCATION", "I-LOCATION", "B-NAME", "I-NAME", "B-PHONE", "I-PHONE", "O"
]
label_to_id = {l: i for i, l in enumerate(target_labels)}
id_to_label = {i: l for l, i in label_to_id.items()}

# -------------------------
# 4) Generate Predictions with Fixed Function
# -------------------------
print("🔄 Generating BioBERT predictions...")
gt_predictions = predict_entities_fixed(gt_tokens, model, tokenizer)
print(f"✅ Generated predictions for {len(gt_predictions)} samples")

# -------------------------
# 5) Safe Evaluation
# -------------------------
print("🔄 Evaluating BioBERT predictions...")
evaluation_results = safe_sequence_metrics(gt_labels_clean, gt_predictions)

# Per-entity evaluation
entity_metrics = {}
entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]

for entity in entities:
    # Filter sequences for this entity
    entity_true = []
    entity_pred = []

    for true_seq, pred_seq in zip(gt_labels_clean, gt_predictions):
        et = [label if entity in label else 'O' for label in true_seq]
        ep = [label if entity in label else 'O' for label in pred_seq]
        entity_true.append(et)
        entity_pred.append(ep)

    entity_result = safe_sequence_metrics(entity_true, entity_pred)
    entity_metrics[entity] = {
        'f1': entity_result['f1'],
        'precision': entity_result['precision'],
        'recall': entity_result['recall']
    }

# -------------------------
# 6) Save Results
# -------------------------
results_data = {
    'model_path': model_path,
    'model_type': 'BioBERT',
    'base_model': 'dmis-lab/biobert-base-cased-v1.1',
    'test_samples': len(gt_tokens),
    'overall_metrics': {
        'f1': evaluation_results['f1'],
        'precision': evaluation_results['precision'],
        'recall': evaluation_results['recall']
    },
    'entity_metrics': entity_metrics,
    'detailed_stats': {
        'true_entities': evaluation_results['true_entities'],
        'pred_entities': evaluation_results['pred_entities'],
        'tp': evaluation_results['tp'],
        'fp': evaluation_results['fp'],
        'fn': evaluation_results['fn']
    },
    'predictions': gt_predictions,
    'true_labels': gt_labels_clean,
    'tokens': gt_tokens
}

# Save results
with open('biobert_evaluation_results.json', 'w') as f:
    json.dump(results_data, f, indent=2)

# Create entity DataFrame
entity_df = pd.DataFrame(entity_metrics).T
entity_df.to_csv('biobert_entity_performance.csv')

print("✅ BioBERT results saved successfully!")

# -------------------------
# 7) Print Summary
# -------------------------
print("\n" + "="*60)
print("🧬 BIOBERT EVALUATION RESULTS")
print("="*60)

print(f"\n📊 Overall Performance:")
print(f"   • F1-Score:  {evaluation_results['f1']:.4f}")
print(f"   • Precision: {evaluation_results['precision']:.4f}")
print(f"   • Recall:    {evaluation_results['recall']:.4f}")

print(f"\n📈 Entity Counts:")
print(f"   • True entities found: {evaluation_results['true_entities']}")
print(f"   • Predicted entities: {evaluation_results['pred_entities']}")
print(f"   • True Positives: {evaluation_results['tp']}")
print(f"   • False Positives: {evaluation_results['fp']}")
print(f"   • False Negatives: {evaluation_results['fn']}")

print(f"\n🎯 Entity-Level Performance:")
for entity, metrics in entity_metrics.items():
    print(f"   • {entity:10s}: F1={metrics['f1']:.3f}, P={metrics['precision']:.3f}, R={metrics['recall']:.3f}")

print(f"\n🧬 BioBERT Model Information:")
print(f"   • Base Model: dmis-lab/biobert-base-cased-v1.1")
print(f"   • Domain: Biomedical Literature (PubMed + PMC)")
print(f"   • Specialization: Medical terminology understanding")
print(f"   • Model Path: {model_path}")

print(f"\n✅ BioBERT evaluation completed successfully!")
print(f"📁 Files saved:")
print(f"   • biobert_evaluation_results.json")
print(f"   • biobert_entity_performance.csv")

🧬 Loading BioBERT trained model and tokenizer...
✅ BioBERT tokenizer loaded
🔍 Trying to load BioBERT model from: ./outputs/biobert_phi_detection
✅ BioBERT model loaded from: ./outputs/biobert_phi_detection
✅ BioBERT model ready on cuda
✅ Data loaded: Test=220, GT=220 samples
🔄 Generating BioBERT predictions...
✅ Generated predictions for 220 samples
🔄 Evaluating BioBERT predictions...
✅ BioBERT results saved successfully!

🧬 BIOBERT EVALUATION RESULTS

📊 Overall Performance:
   • F1-Score:  0.9042
   • Precision: 0.9601
   • Recall:    0.8544

📈 Entity Counts:
   • True entities found: 2899
   • Predicted entities: 2580
   • True Positives: 2477
   • False Positives: 103
   • False Negatives: 422

🎯 Entity-Level Performance:
   • DATE      : F1=0.900, P=0.971, R=0.839
   • HOSPITAL  : F1=0.861, P=0.919, R=0.811
   • ID        : F1=0.983, P=0.995, R=0.972
   • LOCATION  : F1=0.240, P=0.462, R=0.162
   • NAME      : F1=0.787, P=0.882, R=0.711
   • PHONE     : F1=0.875, P=1.000, R=0.778



In [ ]:
# ===========================
# BIOBERT PHI DETECTION MODEL DOWNLOAD
# ===========================

import os
import json
from google.colab import files

print("🧬 Preparing BioBERT PHI Detection model for download...")

# Check what files exist in the outputs directory
if os.path.exists('./outputs'):
    print("✅ Found outputs directory")

    # List contents of outputs directory
    print("\n📂 Contents of ./outputs/:")
    for root, dirs, files_list in os.walk('./outputs'):
        level = root.replace('./outputs', '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files_list:
            file_size = os.path.getsize(os.path.join(root, file))
            file_size_mb = file_size / (1024 * 1024)
            print(f"{subindent}{file} ({file_size_mb:.1f} MB)")
else:
    print("❌ outputs directory not found")

    # Check for alternative locations
    print("\n🔍 Checking alternative model save locations...")

    # Check current directory for biobert files
    current_files = [f for f in os.listdir('.') if 'biobert' in f.lower() or 'model' in f.lower()]
    if current_files:
        print(f"✅ Found model files in current directory: {current_files}")

    # Check if trainer saved model elsewhere
    if 'trainer' in globals():
        print(f"✅ Trainer output directory: {trainer.args.output_dir}")
        if os.path.exists(trainer.args.output_dir):
            print("✅ Trainer output directory exists")
        else:
            print("❌ Trainer output directory not found")

# Method 1: Try BioBERT specific path
if os.path.exists('./outputs/biobert_phi_detection'):
    print("\n📦 Creating zip file from ./outputs/biobert_phi_detection...")
    !zip -rq biobert_phi_detection_model.zip ./outputs/biobert_phi_detection

    if os.path.exists('biobert_phi_detection_model.zip'):
        print("✅ BioBERT zip file created successfully")

        # Check zip file size
        zip_size = os.path.getsize('biobert_phi_detection_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"📦 Zip file size: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting BioBERT model download...")
        files.download('biobert_phi_detection_model.zip')
        print("✅ BioBERT download initiated!")
    else:
        print("❌ Failed to create BioBERT zip file")

# Method 2: Try trainer output directory
elif 'trainer' in globals() and os.path.exists(trainer.args.output_dir):
    print(f"\n📦 Creating zip file from trainer output: {trainer.args.output_dir}")
    !zip -rq biobert_phi_detection_model.zip {trainer.args.output_dir}

    if os.path.exists('biobert_phi_detection_model.zip'):
        print("✅ BioBERT zip file created successfully")

        # Check zip file size
        zip_size = os.path.getsize('biobert_phi_detection_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"📦 Zip file size: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting BioBERT model download...")
        files.download('biobert_phi_detection_model.zip')
        print("✅ BioBERT download initiated!")
    else:
        print("❌ Failed to create BioBERT zip file")

# Method 3: Manual save if trainer exists
elif 'trainer' in globals():
    print("\n💾 Manually saving BioBERT model files...")

    # Create a directory for the BioBERT model
    os.makedirs('biobert_phi_detection_manual', exist_ok=True)

    # Save the model and tokenizer
    trainer.save_model('biobert_phi_detection_manual')
    print("✅ BioBERT model saved")

    if 'tokenizer' in globals():
        tokenizer.save_pretrained('biobert_phi_detection_manual')
        print("✅ BioBERT tokenizer saved")

    # Save BioBERT-specific files

    # Save label mappings
    if 'id_to_label' in globals():
        with open('biobert_phi_detection_manual/id_to_label.json', 'w') as f:
            json.dump(id_to_label, f, indent=2)
        print("✅ PHI label mappings saved")

    if 'label_to_id' in globals():
        with open('biobert_phi_detection_manual/label_to_id.json', 'w') as f:
            json.dump(label_to_id, f, indent=2)

    # Save target labels for PHI detection
    if 'target_labels' in globals():
        with open('biobert_phi_detection_manual/target_labels.json', 'w') as f:
            json.dump(target_labels, f, indent=2)
        print("✅ PHI target labels saved")

    # Save training arguments
    if hasattr(trainer, 'args'):
        trainer.args.save_to_json('biobert_phi_detection_manual/training_args.json')
        print("✅ BioBERT training arguments saved")

    # Save BioBERT results if available
    if os.path.exists('biobert_results.json'):
        !cp biobert_results.json biobert_phi_detection_manual/
        print("✅ BioBERT training results saved")

    # Save model info and usage instructions
    model_info = {
        "model_name": "BioBERT PHI Detection",
        "base_model": "dmis-lab/biobert-base-cased-v1.1",
        "task": "Named Entity Recognition (NER) for PHI Detection",
        "target_entities": [
            "DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"
        ],
        "medical_term_density": globals().get('medical_density', 'N/A'),
        "training_config": {
            "learning_rate": "2e-5",
            "epochs": 8,
            "batch_size": 4,
            "model_type": "BioBERT (biomedical pre-trained)"
        },
        "usage_instructions": {
            "load_model": "AutoModelForTokenClassification.from_pretrained('./biobert_phi_detection')",
            "load_tokenizer": "AutoTokenizer.from_pretrained('./biobert_phi_detection')",
            "preprocessing": "Use is_split_into_words=True for tokenization"
        }
    }

    with open('biobert_phi_detection_manual/model_info.json', 'w') as f:
        json.dump(model_info, f, indent=2)
    print("✅ BioBERT model info saved")

    # Create README file
    readme_content = """# BioBERT PHI Detection Model

## Overview
This is a fine-tuned BioBERT model for detecting Protected Health Information (PHI) in medical texts.

## Base Model
- **Model**: dmis-lab/biobert-base-cased-v1.1
- **Type**: BERT-based model pre-trained on biomedical texts (PubMed and PMC)
- **Task**: Named Entity Recognition (NER)

## Detected PHI Types
- **DATE**: Dates and temporal information
- **HOSPITAL**: Hospital and healthcare facility names
- **ID**: Patient/medical identifiers
- **LOCATION**: Geographic locations
- **NAME**: Person names (patients, doctors)
- **PHONE**: Phone numbers

## Usage

```python
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Load model and tokenizer
model = AutoModelForTokenClassification.from_pretrained('./biobert_phi_detection')
tokenizer = AutoTokenizer.from_pretrained('./biobert_phi_detection')

# Example usage
text = ["Patient", "John", "Smith", "visited", "City", "Hospital"]
inputs = tokenizer(text, is_split_into_words=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Load label mappings
import json
with open('./biobert_phi_detection/id_to_label.json') as f:
    id_to_label = json.load(f)

# Convert predictions to labels
pred_labels = [id_to_label[str(pred.item())] for pred in predictions[0]]
```

## Model Performance
Check `biobert_results.json` for detailed performance metrics.

## BioBERT Advantages
- Pre-trained on biomedical literature (PubMed/PMC)
- Better understanding of medical terminology
- Optimized for healthcare text processing
"""

    with open('biobert_phi_detection_manual/README.md', 'w') as f:
        f.write(readme_content)
    print("✅ README file created")

    # Create zip file
    print("📦 Creating BioBERT model zip file...")
    !zip -rq biobert_phi_detection_model.zip biobert_phi_detection_manual/

    if os.path.exists('biobert_phi_detection_model.zip'):
        zip_size = os.path.getsize('biobert_phi_detection_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"✅ BioBERT zip file created: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting BioBERT model download...")
        files.download('biobert_phi_detection_model.zip')
        print("✅ BioBERT download initiated!")
    else:
        print("❌ Failed to create BioBERT zip file")

else:
    print("❌ No BioBERT model found to download")
    print("\n🔧 BioBERT Troubleshooting steps:")
    print("1. Make sure your BioBERT training completed successfully")
    print("2. Check if trainer.save_model() was called")
    print("3. Verify the output directory path: './outputs/biobert_phi_detection'")
    print("4. Try running: trainer.save_model('./biobert_phi_detection_download')")

print("\n📋 What's included in the BioBERT model zip:")
print("🧬 BioBERT model weights (pytorch_model.bin)")
print("🧬 BioBERT configuration (config.json)")
print("🧬 BioBERT tokenizer files (vocab.txt, tokenizer.json, etc.)")
print("🧬 PHI label mappings (id_to_label.json, label_to_id.json)")
print("🧬 Target PHI labels (target_labels.json)")
print("🧬 Training arguments and results")
print("🧬 Model information and usage guide")
print("🧬 README with setup instructions")

print("\n🚀 Load your BioBERT PHI Detection model later with:")
print("   from transformers import AutoModelForTokenClassification, AutoTokenizer")
print("   model = AutoModelForTokenClassification.from_pretrained('./biobert_phi_detection')")
print("   tokenizer = AutoTokenizer.from_pretrained('./biobert_phi_detection')")

print("\n🧬 BioBERT Advantages:")
print("   ✅ Pre-trained on PubMed and PMC biomedical literature")
print("   ✅ Superior performance on medical text understanding")
print("   ✅ Optimized vocabulary for healthcare terminology")
print("   ✅ Better PHI detection in clinical contexts")

# Clean up temporary files
print("\n🧹 Cleaning up temporary files...")
if os.path.exists('biobert_phi_detection_manual'):
    !rm -rf biobert_phi_detection_manual
    print("✅ Temporary directory cleaned")

print("\n✅ BioBERT PHI Detection model download complete!")

ROBERTA training


In [6]:
# =========================
# ROBERTA-LARGE TRAINING FOR PHI DETECTION - FIXED
# =========================
import random
import numpy as np
import torch
import json
from collections import Counter, defaultdict
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         TrainingArguments, Trainer, EarlyStoppingCallback)
from torch.utils.data import Dataset as TorchDataset
from seqeval.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

print("🤖 Starting RoBERTa-Large Training for PHI Detection (Fixed)")
print("="*60)

# -------------------------
# 1) Setup and Configuration
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# Memory check
def check_gpu_memory():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory // 1024**3
        print(f"🧠 GPU Memory: {total_memory}GB")
        return total_memory
    return 0

gpu_memory = check_gpu_memory()

# Data loading functions
def load_and_validate_data(filepath):
    with open(filepath) as f:
        data = json.load(f)
    return [entry for entry in data if len(entry["tokens"]) == len(entry["labels"])]

def clean_and_normalize_labels(labels):
    out = []
    for l in labels:
        if l in ["B-AGE", "I-AGE"]:
            out.append("O")
        elif l in ["B-DOCTOR", "I-DOCTOR"]:
            out.append(l.replace("DOCTOR", "NAME"))
        elif l in ["B-PATIENT", "I-PATIENT"]:
            out.append(l.replace("PATIENT", "NAME"))
        else:
            out.append(l)
    return out

# Load data
train_data = load_and_validate_data('/content/train.json')
val_data = load_and_validate_data('/content/val.json')

train_tokens = [e["tokens"] for e in train_data]
train_labels = [clean_and_normalize_labels(e["labels"]) for e in train_data]
val_tokens = [e["tokens"] for e in val_data]
val_labels = [clean_and_normalize_labels(e["labels"]) for e in val_data]

print(f"✅ Data loaded - Train: {len(train_tokens)} | Val: {len(val_tokens)}")

# Label setup
target_labels = [
    "B-DATE", "I-DATE", "B-HOSPITAL", "I-HOSPITAL", "B-ID", "I-ID",
    "B-LOCATION", "I-LOCATION", "B-NAME", "I-NAME", "B-PHONE", "I-PHONE", "O"
]
label_to_id = {l: i for i, l in enumerate(target_labels)}
id_to_label = {i: l for l, i in label_to_id.items()}

print(f"📋 Target labels: {len(target_labels)} classes")

# -------------------------
# 2) Fixed RoBERTa Dataset Class
# -------------------------
class RoBERTaNERDataset(TorchDataset):
    def __init__(self, tokens, labels, tokenizer, label_to_id, max_len=256):
        self.tokens = tokens
        self.labels = labels
        self.tokenizer = tokenizer
        self.label_to_id = label_to_id
        self.max_len = max_len

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        words = self.tokens[idx]
        labs = self.labels[idx]

        # Ensure consistency
        min_len = min(len(words), len(labs))
        words = words[:min_len]
        labs = labs[:min_len]

        # Handle long sequences
        if len(words) > self.max_len - 2:
            words = words[:self.max_len - 2]
            labs = labs[:self.max_len - 2]

        # Tokenize with RoBERTa (add_prefix_space handled in tokenizer init)
        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            truncation=True,
            max_length=self.max_len,
            padding="max_length",
            return_tensors="pt"
        )

        # Get word IDs for label alignment
        word_ids = encoding.word_ids()

        # Align labels
        aligned_labels = []
        prev_word_id = None

        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)  # Special tokens
            elif word_id != prev_word_id:
                if word_id < len(labs):
                    label = labs[word_id]
                    aligned_labels.append(self.label_to_id.get(label, self.label_to_id["O"]))
                else:
                    aligned_labels.append(self.label_to_id["O"])
            else:
                aligned_labels.append(-100)  # Subword continuation
            prev_word_id = word_id

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(aligned_labels, dtype=torch.long)
        }

# -------------------------
# 3) Calculate Class Weights
# -------------------------
def calculate_class_weights(labels_list):
    """Calculate balanced class weights"""
    flat_labels = [lbl for seq in labels_list for lbl in seq]
    counts = Counter(flat_labels)
    total = len(flat_labels)

    weights = {}
    for label in target_labels:
        count = counts.get(label, 1)
        if label == "O":
            weights[label] = 0.5  # Lower weight for O class
        else:
            base_weight = total / (len(target_labels) * count)
            weights[label] = min(base_weight * 1.5, 8.0)  # Cap weights

    return weights

# -------------------------
# 4) Metrics Function
# -------------------------
def compute_metrics(eval_pred):
    """Compute F1, precision, recall"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)

    true_labels = []
    pred_labels = []

    for pred_seq, label_seq in zip(predictions, labels):
        true_seq = []
        pred_seq_clean = []

        for pred, label in zip(pred_seq, label_seq):
            if label != -100:
                true_seq.append(id_to_label[label])
                pred_seq_clean.append(id_to_label[pred])

        if true_seq:
            true_labels.append(true_seq)
            pred_labels.append(pred_seq_clean)

    if not true_labels:
        return {"f1": 0.0, "precision": 0.0, "recall": 0.0}

    f1 = f1_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)

    return {
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

# -------------------------
# 5) Custom Trainer with Weighted Loss
# -------------------------
class WeightedRoBERTaTrainer(Trainer):
    def __init__(self, class_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Active loss calculation
        active_loss = labels.view(-1) != -100
        active_logits = logits.view(-1, logits.size(-1))[active_loss]
        active_labels = labels.view(-1)[active_loss]

        if len(active_labels) == 0:
            return torch.tensor(0.0, requires_grad=True, device=logits.device)

        # Weighted cross-entropy
        loss = F.cross_entropy(active_logits, active_labels, weight=self.class_weights)

        return (loss, outputs) if return_outputs else loss

# -------------------------
# 6) Main Training Function
# -------------------------
def train_roberta_large():
    """Train RoBERTa-Large with proper configuration"""

    print("\n🤖 Loading RoBERTa-Large...")

    try:
        # Load tokenizer and model
        model_name = "roberta-large"

        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, add_prefix_space=True)
        print(f"✅ Tokenizer loaded (vocab: {tokenizer.vocab_size})")
        print(f"   • RoBERTa tokenizer configured for pretokenized inputs")

        model = AutoModelForTokenClassification.from_pretrained(
            model_name,
            num_labels=len(target_labels),
            id2label=id_to_label,
            label2id=label_to_id,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1
        )

        print(f"✅ Model loaded (355M parameters)")
        model.to(device)

    except Exception as e:
        print(f"❌ Failed to load RoBERTa-Large: {e}")
        return None, None

    # Calculate class weights
    weight_dict = calculate_class_weights(train_labels)
    class_weights = torch.tensor([weight_dict[label] for label in target_labels],
                               dtype=torch.float).to(device)

    print(f"\n⚖️ Class weights calculated")
    for i, (label, weight) in enumerate(zip(target_labels, class_weights.cpu().numpy())):
        if "LOCATION" in label or "PHONE" in label:
            print(f"   🎯 {label}: {weight:.2f}")

    # Create datasets
    print(f"\n📊 Creating datasets...")
    train_dataset = RoBERTaNERDataset(train_tokens, train_labels, tokenizer, label_to_id)
    val_dataset = RoBERTaNERDataset(val_tokens, val_labels, tokenizer, label_to_id)

    # Configure training based on GPU memory
    if gpu_memory >= 12:
        batch_size = 4
        gradient_accumulation = 4
    elif gpu_memory >= 8:
        batch_size = 2
        gradient_accumulation = 8
    else:
        batch_size = 1
        gradient_accumulation = 16

    print(f"🔧 Training configuration:")
    print(f"   • Batch size: {batch_size}")
    print(f"   • Gradient accumulation: {gradient_accumulation}")
    print(f"   • Effective batch size: {batch_size * gradient_accumulation}")

    # Training arguments - FIXED VERSION
    training_args = TrainingArguments(
        output_dir="./outputs/roberta_large_phi_detection",
        learning_rate=1e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=gradient_accumulation,
        num_train_epochs=4,
        warmup_ratio=0.1,
        weight_decay=0.01,
        max_grad_norm=1.0,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        fp16=True,
        dataloader_pin_memory=False,
        dataloader_num_workers=0,
        seed=SEED,
        logging_steps=50,
        save_total_limit=2,
        report_to="none",
        remove_unused_columns=True,
    )

    # Create trainer
    trainer = WeightedRoBERTaTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        class_weights=class_weights,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    print(f"\n🚀 Starting RoBERTa-Large training...")
    print(f"⏱️ Estimated time: 30-60 minutes")

    try:
        # Clear cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # Train
        train_result = trainer.train()

        # Save model
        trainer.save_model()
        tokenizer.save_pretrained("./outputs/roberta_large_phi_detection")

        print(f"✅ Training completed successfully!")

        # Evaluate
        eval_results = trainer.evaluate()

        print(f"\n📈 RoBERTa-Large Results:")
        print(f"   • F1-Score: {eval_results['eval_f1']:.4f}")
        print(f"   • Precision: {eval_results['eval_precision']:.4f}")
        print(f"   • Recall: {eval_results['eval_recall']:.4f}")

        # Save results
        roberta_results = {
            'model': 'RoBERTa-Large',
            'parameters': '355M',
            'overall_metrics': {
                'f1': eval_results['eval_f1'],
                'precision': eval_results['eval_precision'],
                'recall': eval_results['eval_recall']
            },
            'training_config': {
                'batch_size': batch_size,
                'gradient_accumulation': gradient_accumulation,
                'learning_rate': 1e-5,
                'epochs': 4
            }
        }

        with open('roberta_large_results.json', 'w') as f:
            json.dump(roberta_results, f, indent=2)

        print(f"📁 Model saved to: ./outputs/roberta_large_phi_detection")
        print(f"📊 Results saved to: roberta_large_results.json")

        return trainer, eval_results

    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            print(f"❌ GPU out of memory")
            print(f"💡 Try reducing batch_size to 1 or max_length to 128")
        else:
            print(f"❌ Training error: {e}")

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return None, None

    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None, None

# -------------------------
# 7) Execute Training
# -------------------------
if __name__ == "__main__":
    print(f"\n🎯 Starting RoBERTa-Large PHI Detection Training")

    try:
        trainer, results = train_roberta_large()

        if trainer is not None and results is not None:
            print(f"\n🎉 RoBERTa-Large Training Successful!")
            print(f"🚀 Ready for comparison with BioBERT and ClinicalBERT!")
        else:
            print(f"\n⚠️ RoBERTa-Large training encountered issues")
            print(f"💡 Check GPU memory and try reducing batch size")

    except Exception as e:
        print(f"❌ Training failed: {e}")

    finally:
        # Clean up
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print(f"\n✅ Training process completed!")

🤖 Starting RoBERTa-Large Training for PHI Detection (Fixed)
✅ Using device: cuda
🧠 GPU Memory: 14GB
✅ Data loaded - Train: 669 | Val: 220
📋 Target labels: 13 classes

🎯 Starting RoBERTa-Large PHI Detection Training

🤖 Loading RoBERTa-Large...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded (355M parameters)

⚖️ Class weights calculated
   🎯 B-LOCATION: 8.00
   🎯 I-LOCATION: 8.00
   🎯 B-PHONE: 8.00
   🎯 I-PHONE: 8.00

📊 Creating datasets...
🔧 Training configuration:
   • Batch size: 4
   • Gradient accumulation: 4
   • Effective batch size: 16

🚀 Starting RoBERTa-Large training...
⏱️ Estimated time: 30-60 minutes


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,No log,0.178089,0.871679,0.828735,0.919316
2,1.113200,0.070108,0.968425,0.955865,0.981320
3,0.058700,0.048700,0.973601,0.965234,0.982114
4,0.026600,0.041203,0.973016,0.964467,0.981717


✅ Training completed successfully!



📈 RoBERTa-Large Results:
   • F1-Score: 0.9736
   • Precision: 0.9652
   • Recall: 0.9821
📁 Model saved to: ./outputs/roberta_large_phi_detection
📊 Results saved to: roberta_large_results.json

🎉 RoBERTa-Large Training Successful!
🚀 Ready for comparison with BioBERT and ClinicalBERT!

✅ Training process completed!


In [7]:
# =========================
# FIXED RoBERTa-Large Evaluation Suite
# =========================
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter, defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# 1) Safe Evaluation Functions (Handle Length Mismatches)
# -------------------------
def safe_sequence_metrics(true_labels, pred_labels):
    """Calculate metrics safely handling sequence length mismatches"""

    # Ensure sequences have same length
    aligned_true = []
    aligned_pred = []

    for true_seq, pred_seq in zip(true_labels, pred_labels):
        min_len = min(len(true_seq), len(pred_seq))
        if min_len > 0:
            aligned_true.append(true_seq[:min_len])
            aligned_pred.append(pred_seq[:min_len])

    if not aligned_true:
        return {"f1": 0.0, "precision": 0.0, "recall": 0.0}

    # Calculate token-level metrics
    all_true_flat = [label for seq in aligned_true for label in seq]
    all_pred_flat = [label for seq in aligned_pred for label in seq]

    # Entity-level metrics (safer approach)
    true_entities = set()
    pred_entities = set()

    for seq_idx, (true_seq, pred_seq) in enumerate(zip(aligned_true, aligned_pred)):
        # Extract entities from true sequence
        current_entity = None
        entity_start = None

        for pos, label in enumerate(true_seq):
            if label.startswith('B-'):
                if current_entity is not None:
                    true_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = label[2:]
                entity_start = pos
            elif label.startswith('I-') and current_entity is not None:
                continue  # Continue current entity
            else:  # 'O' or different entity
                if current_entity is not None:
                    true_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = None
                entity_start = None

        # Handle entity at end of sequence
        if current_entity is not None:
            true_entities.add((seq_idx, entity_start, len(true_seq)-1, current_entity))

        # Extract entities from predicted sequence
        current_entity = None
        entity_start = None

        for pos, label in enumerate(pred_seq):
            if label.startswith('B-'):
                if current_entity is not None:
                    pred_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = label[2:]
                entity_start = pos
            elif label.startswith('I-') and current_entity is not None:
                continue
            else:
                if current_entity is not None:
                    pred_entities.add((seq_idx, entity_start, pos-1, current_entity))
                current_entity = None
                entity_start = None

        if current_entity is not None:
            pred_entities.add((seq_idx, entity_start, len(pred_seq)-1, current_entity))

    # Calculate metrics
    tp = len(true_entities & pred_entities)
    fp = len(pred_entities - true_entities)
    fn = len(true_entities - pred_entities)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "true_entities": len(true_entities),
        "pred_entities": len(pred_entities),
        "tp": tp,
        "fp": fp,
        "fn": fn
    }

# -------------------------
# 2) Fixed Prediction Function
# -------------------------
def predict_entities_fixed(tokens_list, model, tokenizer, max_len=256):
    """Generate predictions with better alignment"""
    predictions = []

    model.eval()
    with torch.no_grad():
        for tokens in tokens_list:
            try:
                # Limit sequence length
                if len(tokens) > max_len - 2:
                    tokens = tokens[:max_len - 2]

                # Tokenize (RoBERTa specific with add_prefix_space)
                encoding = tokenizer(
                    tokens,
                    is_split_into_words=True,
                    truncation=True,
                    max_length=max_len,
                    padding="max_length",
                    return_tensors="pt"
                )

                # Get word IDs for alignment
                word_ids = encoding.word_ids()

                # Move to device
                device = next(model.parameters()).device
                encoding = {k: v.to(device) for k, v in encoding.items()}

                # Predict
                outputs = model(**encoding)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1).cpu().numpy()[0]

                # Align predictions with original tokens
                aligned_preds = []
                prev_word_id = None

                for i, word_id in enumerate(word_ids):
                    if word_id is None:
                        continue  # Skip special tokens
                    elif word_id != prev_word_id:
                        # First subword of a word
                        if word_id < len(tokens):
                            pred_id = preds[i]
                            pred_label = id_to_label.get(pred_id, "O")
                            aligned_preds.append(pred_label)
                    prev_word_id = word_id

                # Ensure same length as input tokens
                while len(aligned_preds) < len(tokens):
                    aligned_preds.append("O")
                aligned_preds = aligned_preds[:len(tokens)]

                predictions.append(aligned_preds)

            except Exception as e:
                print(f"Error processing sequence: {e}")
                # Fallback: all O labels
                predictions.append(["O"] * len(tokens))

    return predictions

# -------------------------
# 3) Load Model and Data
# -------------------------
print("🤖 Loading RoBERTa-Large trained model and tokenizer...")

# Try multiple possible RoBERTa model paths
possible_paths = [
    "./outputs/roberta_large_phi_detection",
    "/content/outputs/roberta_large_phi_detection",
    "./roberta_large_phi_detection",
    "/content/roberta_large_phi_detection"
]

model = None
model_path = None

try:
    # Load RoBERTa tokenizer with add_prefix_space=True
    tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True, add_prefix_space=True)
    print("✅ RoBERTa-Large tokenizer loaded")

    # Try to find the model
    for path in possible_paths:
        try:
            print(f"🔍 Trying to load RoBERTa-Large model from: {path}")
            model = AutoModelForTokenClassification.from_pretrained(path)
            model_path = path
            print(f"✅ RoBERTa-Large model loaded from: {path}")
            break
        except Exception as e:
            print(f"❌ Failed to load from {path}: {str(e)[:50]}...")
            continue

    if model is None:
        print("❌ Could not find trained RoBERTa-Large model. Please check if training completed successfully.")
        print("💡 Available files in current directory:")
        import os
        if os.path.exists("outputs"):
            print(f"   outputs/ contents: {os.listdir('outputs')}")
        else:
            print("   No outputs/ directory found")
        exit()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print(f"✅ RoBERTa-Large model ready on {device}")

except Exception as e:
    print(f"❌ Error loading RoBERTa-Large model: {e}")
    exit()

# Load data
try:
    with open('/content/test.json') as f:
        test_data = json.load(f)
    test_tokens = [entry["tokens"] for entry in test_data]

    with open('/content/val.json') as f:
        gt_data = json.load(f)
    gt_tokens = [entry["tokens"] for entry in gt_data]
    gt_labels = [entry["labels"] for entry in gt_data]

    print(f"✅ Data loaded: Test={len(test_tokens)}, GT={len(gt_tokens)} samples")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    exit()

# Clean labels (same as training)
def clean_and_normalize_labels(labels):
    out = []
    for l in labels:
        if l in ["B-AGE", "I-AGE"]:
            out.append("O")
        elif l in ["B-DOCTOR", "I-DOCTOR"]:
            out.append(l.replace("DOCTOR", "NAME"))
        elif l in ["B-PATIENT", "I-PATIENT"]:
            out.append(l.replace("PATIENT", "NAME"))
        else:
            out.append(l)
    return out

gt_labels_clean = [clean_and_normalize_labels(labels) for labels in gt_labels]

# Label mappings
target_labels = [
    "B-DATE", "I-DATE", "B-HOSPITAL", "I-HOSPITAL", "B-ID", "I-ID",
    "B-LOCATION", "I-LOCATION", "B-NAME", "I-NAME", "B-PHONE", "I-PHONE", "O"
]
label_to_id = {l: i for i, l in enumerate(target_labels)}
id_to_label = {i: l for l, i in label_to_id.items()}

# -------------------------
# 4) Generate Predictions with Fixed Function
# -------------------------
print("🔄 Generating RoBERTa-Large predictions...")
gt_predictions = predict_entities_fixed(gt_tokens, model, tokenizer)
print(f"✅ Generated predictions for {len(gt_predictions)} samples")

# -------------------------
# 5) Safe Evaluation
# -------------------------
print("🔄 Evaluating RoBERTa-Large predictions...")
evaluation_results = safe_sequence_metrics(gt_labels_clean, gt_predictions)

# Per-entity evaluation
entity_metrics = {}
entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]

for entity in entities:
    # Filter sequences for this entity
    entity_true = []
    entity_pred = []

    for true_seq, pred_seq in zip(gt_labels_clean, gt_predictions):
        et = [label if entity in label else 'O' for label in true_seq]
        ep = [label if entity in label else 'O' for label in pred_seq]
        entity_true.append(et)
        entity_pred.append(ep)

    entity_result = safe_sequence_metrics(entity_true, entity_pred)
    entity_metrics[entity] = {
        'f1': entity_result['f1'],
        'precision': entity_result['precision'],
        'recall': entity_result['recall']
    }

# -------------------------
# 6) Save Results
# -------------------------
results_data = {
    'model_path': model_path,
    'model_type': 'RoBERTa-Large',
    'base_model': 'roberta-large',
    'parameters': '355M',
    'test_samples': len(gt_tokens),
    'overall_metrics': {
        'f1': evaluation_results['f1'],
        'precision': evaluation_results['precision'],
        'recall': evaluation_results['recall']
    },
    'entity_metrics': entity_metrics,
    'detailed_stats': {
        'true_entities': evaluation_results['true_entities'],
        'pred_entities': evaluation_results['pred_entities'],
        'tp': evaluation_results['tp'],
        'fp': evaluation_results['fp'],
        'fn': evaluation_results['fn']
    },
    'predictions': gt_predictions,
    'true_labels': gt_labels_clean,
    'tokens': gt_tokens
}

# Save results
with open('roberta_large_evaluation_results.json', 'w') as f:
    json.dump(results_data, f, indent=2)

# Create entity DataFrame
entity_df = pd.DataFrame(entity_metrics).T
entity_df.to_csv('roberta_large_entity_performance.csv')

print("✅ RoBERTa-Large results saved successfully!")

# -------------------------
# 7) Print Summary
# -------------------------
print("\n" + "="*60)
print("🤖 ROBERTA-LARGE EVALUATION RESULTS")
print("="*60)

print(f"\n📊 Overall Performance:")
print(f"   • F1-Score:  {evaluation_results['f1']:.4f}")
print(f"   • Precision: {evaluation_results['precision']:.4f}")
print(f"   • Recall:    {evaluation_results['recall']:.4f}")

print(f"\n📈 Entity Counts:")
print(f"   • True entities found: {evaluation_results['true_entities']}")
print(f"   • Predicted entities: {evaluation_results['pred_entities']}")
print(f"   • True Positives: {evaluation_results['tp']}")
print(f"   • False Positives: {evaluation_results['fp']}")
print(f"   • False Negatives: {evaluation_results['fn']}")

print(f"\n🎯 Entity-Level Performance:")
for entity, metrics in entity_metrics.items():
    print(f"   • {entity:10s}: F1={metrics['f1']:.3f}, P={metrics['precision']:.3f}, R={metrics['recall']:.3f}")

print(f"\n🤖 RoBERTa-Large Model Information:")
print(f"   • Base Model: roberta-large")
print(f"   • Parameters: 355M")
print(f"   • Domain: General Text (BookCorpus + Wikipedia)")
print(f"   • Architecture: RoBERTa (Robustly Optimized BERT)")
print(f"   • Specialization: Advanced transformer with improved training")
print(f"   • Model Path: {model_path}")

print(f"\n🤖 RoBERTa-Large Advantages:")
print(f"   • Large parameter count (355M) for complex pattern recognition")
print(f"   • Improved training methodology over BERT")
print(f"   • Dynamic masking and larger batch sizes")
print(f"   • Robust optimization techniques")
print(f"   • Advanced attention mechanisms")

print(f"\n✅ RoBERTa-Large evaluation completed successfully!")
print(f"📁 Files saved:")
print(f"   • roberta_large_evaluation_results.json")
print(f"   • roberta_large_entity_performance.csv")

🤖 Loading RoBERTa-Large trained model and tokenizer...
✅ RoBERTa-Large tokenizer loaded
🔍 Trying to load RoBERTa-Large model from: ./outputs/roberta_large_phi_detection
✅ RoBERTa-Large model loaded from: ./outputs/roberta_large_phi_detection
✅ RoBERTa-Large model ready on cuda
✅ Data loaded: Test=220, GT=220 samples
🔄 Generating RoBERTa-Large predictions...
✅ Generated predictions for 220 samples
🔄 Evaluating RoBERTa-Large predictions...
✅ RoBERTa-Large results saved successfully!

🤖 ROBERTA-LARGE EVALUATION RESULTS

📊 Overall Performance:
   • F1-Score:  0.9060
   • Precision: 0.9701
   • Recall:    0.8499

📈 Entity Counts:
   • True entities found: 2899
   • Predicted entities: 2540
   • True Positives: 2464
   • False Positives: 76
   • False Negatives: 435

🎯 Entity-Level Performance:
   • DATE      : F1=0.906, P=0.985, R=0.838
   • HOSPITAL  : F1=0.855, P=0.898, R=0.815
   • ID        : F1=0.981, P=0.994, R=0.968
   • LOCATION  : F1=0.000, P=0.000, R=0.000
   • NAME      : F1=0.81

In [ ]:
# ===========================
# ROBERTA-LARGE PHI DETECTION MODEL DOWNLOAD
# ===========================

import os
import json
from google.colab import files

print("🤖 Preparing RoBERTa-Large PHI Detection model for download...")

# Check what files exist in the outputs directory
if os.path.exists('./outputs'):
    print("✅ Found outputs directory")

    # List contents of outputs directory
    print("\n📂 Contents of ./outputs/:")
    for root, dirs, files_list in os.walk('./outputs'):
        level = root.replace('./outputs', '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files_list:
            file_size = os.path.getsize(os.path.join(root, file))
            file_size_mb = file_size / (1024 * 1024)
            print(f"{subindent}{file} ({file_size_mb:.1f} MB)")
else:
    print("❌ outputs directory not found")

    # Check for alternative locations
    print("\n🔍 Checking alternative model save locations...")

    # Check current directory for roberta files
    current_files = [f for f in os.listdir('.') if 'roberta' in f.lower() or 'model' in f.lower()]
    if current_files:
        print(f"✅ Found model files in current directory: {current_files}")

    # Check if trainer saved model elsewhere
    if 'trainer' in globals():
        print(f"✅ Trainer output directory: {trainer.args.output_dir}")
        if os.path.exists(trainer.args.output_dir):
            print("✅ Trainer output directory exists")
        else:
            print("❌ Trainer output directory not found")

# Method 1: Try RoBERTa specific path
if os.path.exists('./outputs/roberta_large_phi_detection'):
    print("\n📦 Creating zip file from ./outputs/roberta_large_phi_detection...")
    !zip -rq roberta_large_phi_detection_model.zip ./outputs/roberta_large_phi_detection

    if os.path.exists('roberta_large_phi_detection_model.zip'):
        print("✅ RoBERTa-Large zip file created successfully")

        # Check zip file size
        zip_size = os.path.getsize('roberta_large_phi_detection_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"📦 Zip file size: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting RoBERTa-Large model download...")
        files.download('roberta_large_phi_detection_model.zip')
        print("✅ RoBERTa-Large download initiated!")
    else:
        print("❌ Failed to create RoBERTa-Large zip file")

# Method 2: Try trainer output directory
elif 'trainer' in globals() and os.path.exists(trainer.args.output_dir):
    print(f"\n📦 Creating zip file from trainer output: {trainer.args.output_dir}")
    !zip -rq roberta_large_phi_detection_model.zip {trainer.args.output_dir}

    if os.path.exists('roberta_large_phi_detection_model.zip'):
        print("✅ RoBERTa-Large zip file created successfully")

        # Check zip file size
        zip_size = os.path.getsize('roberta_large_phi_detection_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"📦 Zip file size: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting RoBERTa-Large model download...")
        files.download('roberta_large_phi_detection_model.zip')
        print("✅ RoBERTa-Large download initiated!")
    else:
        print("❌ Failed to create RoBERTa-Large zip file")

# Method 3: Manual save if trainer exists
elif 'trainer' in globals():
    print("\n💾 Manually saving RoBERTa-Large model files...")

    # Create a directory for the RoBERTa model
    os.makedirs('roberta_large_phi_detection_manual', exist_ok=True)

    # Save the model and tokenizer
    trainer.save_model('roberta_large_phi_detection_manual')
    print("✅ RoBERTa-Large model saved")

    if 'tokenizer' in globals():
        tokenizer.save_pretrained('roberta_large_phi_detection_manual')
        print("✅ RoBERTa-Large tokenizer saved")

    # Save RoBERTa-specific files

    # Save label mappings
    if 'id_to_label' in globals():
        with open('roberta_large_phi_detection_manual/id_to_label.json', 'w') as f:
            json.dump(id_to_label, f, indent=2)
        print("✅ PHI label mappings saved")

    if 'label_to_id' in globals():
        with open('roberta_large_phi_detection_manual/label_to_id.json', 'w') as f:
            json.dump(label_to_id, f, indent=2)

    # Save target labels for PHI detection
    if 'target_labels' in globals():
        with open('roberta_large_phi_detection_manual/target_labels.json', 'w') as f:
            json.dump(target_labels, f, indent=2)
        print("✅ PHI target labels saved")

    # Save training arguments
    if hasattr(trainer, 'args'):
        trainer.args.save_to_json('roberta_large_phi_detection_manual/training_args.json')
        print("✅ RoBERTa-Large training arguments saved")

    # Save RoBERTa results if available
    if os.path.exists('roberta_large_results.json'):
        !cp roberta_large_results.json roberta_large_phi_detection_manual/
        print("✅ RoBERTa-Large training results saved")

    # Save GPU memory info if available
    gpu_info = {}
    if 'gpu_memory' in globals():
        gpu_info['gpu_memory_gb'] = gpu_memory
    if 'batch_size' in locals() or 'batch_size' in globals():
        gpu_info['optimal_batch_size'] = globals().get('batch_size', 'Unknown')
    if 'gradient_accumulation' in locals() or 'gradient_accumulation' in globals():
        gpu_info['gradient_accumulation_steps'] = globals().get('gradient_accumulation', 'Unknown')

    if gpu_info:
        with open('roberta_large_phi_detection_manual/gpu_optimization.json', 'w') as f:
            json.dump(gpu_info, f, indent=2)
        print("✅ GPU optimization settings saved")

    # Save model info and usage instructions
    model_info = {
        "model_name": "RoBERTa-Large PHI Detection",
        "base_model": "roberta-large",
        "parameters": "355M",
        "task": "Named Entity Recognition (NER) for PHI Detection",
        "target_entities": [
            "DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"
        ],
        "training_config": {
            "learning_rate": "1e-5",
            "epochs": 4,
            "effective_batch_size": "Variable based on GPU memory",
            "model_type": "RoBERTa-Large (355M parameters)"
        },
        "advantages": [
            "Large parameter count (355M) for complex pattern recognition",
            "Robustly optimized BERT pretraining approach",
            "Superior performance on downstream NLP tasks",
            "Advanced attention mechanisms"
        ],
        "usage_instructions": {
            "load_model": "AutoModelForTokenClassification.from_pretrained('./roberta_large_phi_detection')",
            "load_tokenizer": "AutoTokenizer.from_pretrained('./roberta_large_phi_detection', add_prefix_space=True)",
            "preprocessing": "Use is_split_into_words=True and add_prefix_space=True for tokenization",
            "memory_requirements": "Requires substantial GPU memory for training and inference"
        }
    }

    with open('roberta_large_phi_detection_manual/model_info.json', 'w') as f:
        json.dump(model_info, f, indent=2)
    print("✅ RoBERTa-Large model info saved")

    # Create README file
    readme_content = """# RoBERTa-Large PHI Detection Model

## Overview
This is a fine-tuned RoBERTa-Large model (355M parameters) for detecting Protected Health Information (PHI) in medical texts.

## Base Model
- **Model**: roberta-large
- **Parameters**: 355M
- **Type**: RoBERTa (Robustly Optimized BERT Pretraining Approach)
- **Task**: Named Entity Recognition (NER)

## Detected PHI Types
- **DATE**: Dates and temporal information
- **HOSPITAL**: Hospital and healthcare facility names
- **ID**: Patient/medical identifiers
- **LOCATION**: Geographic locations
- **NAME**: Person names (patients, doctors)
- **PHONE**: Phone numbers

## Model Advantages
- **Large Scale**: 355M parameters for complex pattern recognition
- **Robust Training**: RoBERTa's improved training methodology
- **High Performance**: Superior results on downstream NLP tasks
- **Advanced Architecture**: State-of-the-art transformer design

## Usage

```python
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Load model and tokenizer (Note: add_prefix_space=True for RoBERTa)
model = AutoModelForTokenClassification.from_pretrained('./roberta_large_phi_detection')
tokenizer = AutoTokenizer.from_pretrained('./roberta_large_phi_detection', add_prefix_space=True)

# Example usage
text = ["Patient", "John", "Smith", "visited", "City", "Hospital"]
inputs = tokenizer(text, is_split_into_words=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Load label mappings
import json
with open('./roberta_large_phi_detection/id_to_label.json') as f:
    id_to_label = json.load(f)

# Convert predictions to labels
pred_labels = [id_to_label[str(pred.item())] for pred in predictions[0]]
```

## Memory Requirements
- **Training**: Requires high-end GPU (12GB+ recommended)
- **Inference**: Can run on smaller GPUs with reduced batch sizes
- **Optimization**: Gradient accumulation used for memory efficiency

## Performance
Check `roberta_large_results.json` for detailed performance metrics.

## GPU Optimization
Check `gpu_optimization.json` for optimal batch sizes and settings based on your hardware.

## RoBERTa Advantages
- Improved training methodology over BERT
- Better handling of subword tokenization
- Enhanced performance on complex NLP tasks
- Robust optimization techniques
"""

    with open('roberta_large_phi_detection_manual/README.md', 'w') as f:
        f.write(readme_content)
    print("✅ README file created")

    # Create zip file
    print("📦 Creating RoBERTa-Large model zip file...")
    !zip -rq roberta_large_phi_detection_model.zip roberta_large_phi_detection_manual/

    if os.path.exists('roberta_large_phi_detection_model.zip'):
        zip_size = os.path.getsize('roberta_large_phi_detection_model.zip')
        zip_size_mb = zip_size / (1024 * 1024)
        print(f"✅ RoBERTa-Large zip file created: {zip_size_mb:.1f} MB")

        # Download the file
        print("⬇️ Starting RoBERTa-Large model download...")
        files.download('roberta_large_phi_detection_model.zip')
        print("✅ RoBERTa-Large download initiated!")
    else:
        print("❌ Failed to create RoBERTa-Large zip file")

else:
    print("❌ No RoBERTa-Large model found to download")
    print("\n🔧 RoBERTa-Large Troubleshooting steps:")
    print("1. Make sure your RoBERTa-Large training completed successfully")
    print("2. Check if trainer.save_model() was called")
    print("3. Verify the output directory path: './outputs/roberta_large_phi_detection'")
    print("4. Try running: trainer.save_model('./roberta_large_phi_detection_download')")

print("\n📋 What's included in the RoBERTa-Large model zip:")
print("🤖 RoBERTa-Large model weights (pytorch_model.bin) - 355M parameters")
print("🤖 RoBERTa-Large configuration (config.json)")
print("🤖 RoBERTa tokenizer files (vocab.json, merges.txt, tokenizer.json, etc.)")
print("🤖 PHI label mappings (id_to_label.json, label_to_id.json)")
print("🤖 Target PHI labels (target_labels.json)")
print("🤖 Training arguments and results")
print("🤖 GPU optimization settings")
print("🤖 Model information and usage guide")
print("🤖 README with setup instructions")

print("\n🚀 Load your RoBERTa-Large PHI Detection model later with:")
print("   from transformers import AutoModelForTokenClassification, AutoTokenizer")
print("   model = AutoModelForTokenClassification.from_pretrained('./roberta_large_phi_detection')")
print("   tokenizer = AutoTokenizer.from_pretrained('./roberta_large_phi_detection', add_prefix_space=True)")

print("\n🤖 RoBERTa-Large Advantages:")
print("   ✅ 355M parameters for superior pattern recognition")
print("   ✅ Robustly optimized BERT pretraining approach")
print("   ✅ State-of-the-art transformer architecture")
print("   ✅ Excellent performance on complex NLP tasks")
print("   ✅ Advanced attention mechanisms")

# Clean up temporary files
print("\n🧹 Cleaning up temporary files...")
if os.path.exists('roberta_large_phi_detection_manual'):
    !rm -rf roberta_large_phi_detection_manual
    print("✅ Temporary directory cleaned")

print("\n✅ RoBERTa-Large PHI Detection model download complete!")

In [ ]:
# =========================
# COMPREHENSIVE PHI MODEL COMPARISON & VISUALIZATION
# =========================
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

print("🔬 Comprehensive PHI Detection Model Comparison")
print("="*60)

# -------------------------
# 1) Load All Model Results
# -------------------------
def load_all_model_results():
    """Load results from all three models"""

    models_data = {}

    # Try to load ClinicalBERT results
    try:
        with open('evaluation_results_clincalbert.json', 'r') as f:
            clinical_data = json.load(f)
        models_data['ClinicalBERT'] = {
            'name': 'ClinicalBERT Enhanced',
            'type': 'Clinical BERT',
            'domain': 'Clinical Notes (MIMIC-III)',
            'parameters': '110M',
            'base_model': 'emilyalsentzer/Bio_ClinicalBERT',
            'overall': clinical_data['overall_metrics'],
            'entities': clinical_data['entity_metrics'],
            'details': clinical_data['detailed_stats'],
            'color': '#1f77b4'
        }
        print("✅ ClinicalBERT results loaded")
    except FileNotFoundError:
        print("⚠️ ClinicalBERT results not found")

    # Try to load BioBERT results
    try:
        with open('biobert_evaluation_results.json', 'r') as f:
            biobert_data = json.load(f)
        models_data['BioBERT'] = {
            'name': 'BioBERT',
            'type': 'Biomedical BERT',
            'domain': 'Biomedical Literature (PubMed + PMC)',
            'parameters': '110M',
            'base_model': 'dmis-lab/biobert-base-cased-v1.1',
            'overall': biobert_data['overall_metrics'],
            'entities': biobert_data['entity_metrics'],
            'details': biobert_data['detailed_stats'],
            'color': '#ff7f0e'
        }
        print("✅ BioBERT results loaded")
    except FileNotFoundError:
        print("⚠️ BioBERT results not found")

    # Try to load RoBERTa results
    try:
        with open('roberta_large_evaluation_results.json', 'r') as f:
            roberta_data = json.load(f)
        models_data['RoBERTa-Large'] = {
            'name': 'RoBERTa-Large',
            'type': 'General Transformer',
            'domain': 'General Text (BookCorpus + Wikipedia)',
            'parameters': '355M',
            'base_model': 'roberta-large',
            'overall': roberta_data['overall_metrics'],
            'entities': roberta_data['entity_metrics'],
            'details': roberta_data['detailed_stats'],
            'color': '#2ca02c'
        }
        print("✅ RoBERTa-Large results loaded")
    except FileNotFoundError:
        print("⚠️ RoBERTa-Large results not found")

    return models_data

# -------------------------
# 2) Create Comprehensive Comparison DataFrames
# -------------------------
def create_comparison_dataframes(models_data):
    """Create detailed comparison DataFrames"""

    if not models_data:
        print("❌ No model data available for comparison")
        return None, None

    # Overall performance comparison
    overall_data = []
    for model_key, model_info in models_data.items():
        overall_data.append({
            'Model': model_info['name'],
            'Type': model_info['type'],
            'Domain': model_info['domain'],
            'Parameters': model_info['parameters'],
            'F1-Score': model_info['overall']['f1'],
            'Precision': model_info['overall']['precision'],
            'Recall': model_info['overall']['recall'],
            'True Entities': model_info['details']['true_entities'],
            'Predicted Entities': model_info['details']['pred_entities'],
            'True Positives': model_info['details']['tp'],
            'False Positives': model_info['details']['fp'],
            'False Negatives': model_info['details']['fn']
        })

    overall_df = pd.DataFrame(overall_data)

    # Entity-level comparison
    entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]
    entity_data = []

    for entity in entities:
        row = {'Entity': entity}
        for model_key, model_info in models_data.items():
            model_name = model_info['name']
            if entity in model_info['entities']:
                row[f'{model_name}_F1'] = model_info['entities'][entity]['f1']
                row[f'{model_name}_Precision'] = model_info['entities'][entity]['precision']
                row[f'{model_name}_Recall'] = model_info['entities'][entity]['recall']
            else:
                row[f'{model_name}_F1'] = 0.0
                row[f'{model_name}_Precision'] = 0.0
                row[f'{model_name}_Recall'] = 0.0
        entity_data.append(row)

    entity_df = pd.DataFrame(entity_data)

    return overall_df, entity_df

# -------------------------
# 3) Advanced Statistical Analysis
# -------------------------
def perform_statistical_analysis(models_data):
    """Perform detailed statistical analysis"""

    print("\n📊 Statistical Analysis Results:")
    print("="*50)

    if len(models_data) < 2:
        print("⚠️ Need at least 2 models for comparison")
        return

    # Performance ranking
    f1_scores = [(name, info['overall']['f1']) for name, info in models_data.items()]
    f1_scores.sort(key=lambda x: x[1], reverse=True)

    print("\n🏆 Overall Performance Ranking (F1-Score):")
    for i, (model, f1) in enumerate(f1_scores, 1):
        print(f"   {i}. {model:20s}: {f1:.4f}")

    # Statistical significance (simplified)
    best_model = f1_scores[0][0]
    best_f1 = f1_scores[0][1]

    print(f"\n📈 Performance Gaps:")
    for model, f1 in f1_scores[1:]:
        gap = best_f1 - f1
        percentage_gap = (gap / best_f1) * 100
        print(f"   • {model}: -{gap:.4f} ({percentage_gap:.1f}% lower than best)")

    # Entity-level analysis
    entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]
    entity_winners = {entity: [] for entity in entities}

    for entity in entities:
        entity_scores = []
        for model_name, model_info in models_data.items():
            if entity in model_info['entities']:
                entity_scores.append((model_name, model_info['entities'][entity]['f1']))

        if entity_scores:
            entity_scores.sort(key=lambda x: x[1], reverse=True)
            entity_winners[entity] = entity_scores

    print(f"\n🎯 Entity-Level Winners:")
    for entity, scores in entity_winners.items():
        if scores:
            winner = scores[0]
            print(f"   • {entity:10s}: {winner[0]} ({winner[1]:.3f})")

    # Model strengths analysis
    print(f"\n💪 Model Strengths Analysis:")
    for model_name, model_info in models_data.items():
        strengths = []
        for entity in entities:
            if entity in model_info['entities']:
                f1 = model_info['entities'][entity]['f1']
                if f1 > 0.7:  # Good performance threshold
                    strengths.append(f"{entity}({f1:.2f})")

        if strengths:
            print(f"   • {model_name}: {', '.join(strengths)}")
        else:
            print(f"   • {model_name}: No entities above 0.7 F1")

# -------------------------
# 4) Create Static Visualizations
# -------------------------
def create_static_visualizations(models_data, overall_df, entity_df):
    """Create comprehensive static visualizations"""

    if not models_data:
        return

    # Set style
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")

    # Create figure with subplots
    fig = plt.figure(figsize=(20, 16))

    # 1. Overall Performance Comparison
    ax1 = plt.subplot(3, 3, 1)
    models = [info['name'] for info in models_data.values()]
    f1_scores = [info['overall']['f1'] for info in models_data.values()]
    precision_scores = [info['overall']['precision'] for info in models_data.values()]
    recall_scores = [info['overall']['recall'] for info in models_data.values()]
    colors = [info['color'] for info in models_data.values()]

    x = np.arange(len(models))
    width = 0.25

    bars1 = ax1.bar(x - width, f1_scores, width, label='F1-Score', alpha=0.8)
    bars2 = ax1.bar(x, precision_scores, width, label='Precision', alpha=0.8)
    bars3 = ax1.bar(x + width, recall_scores, width, label='Recall', alpha=0.8)

    ax1.set_xlabel('Models')
    ax1.set_ylabel('Score')
    ax1.set_title('Overall Performance Metrics', fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(models, rotation=15)
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Add value labels
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=8)

    # 2. Entity-Level F1 Comparison Heatmap
    ax2 = plt.subplot(3, 3, 2)
    entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]
    entity_matrix = []

    for model_name, model_info in models_data.items():
        row = []
        for entity in entities:
            if entity in model_info['entities']:
                row.append(model_info['entities'][entity]['f1'])
            else:
                row.append(0.0)
        entity_matrix.append(row)

    entity_matrix = np.array(entity_matrix)
    im = ax2.imshow(entity_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

    ax2.set_xticks(np.arange(len(entities)))
    ax2.set_yticks(np.arange(len(models)))
    ax2.set_xticklabels(entities, rotation=45)
    ax2.set_yticklabels(models)
    ax2.set_title('Entity-Level F1 Scores Heatmap', fontweight='bold')

    # Add text annotations
    for i in range(len(models)):
        for j in range(len(entities)):
            text = ax2.text(j, i, f'{entity_matrix[i, j]:.2f}',
                           ha="center", va="center", color="black", fontweight='bold')

    plt.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)

    # 3. Model Parameter Comparison
    ax3 = plt.subplot(3, 3, 3)
    params = [355 if '355M' in info['parameters'] else 110 for info in models_data.values()]
    bars = ax3.bar(models, params, color=colors, alpha=0.7)
    ax3.set_ylabel('Parameters (Millions)')
    ax3.set_title('Model Size Comparison', fontweight='bold')
    ax3.tick_params(axis='x', rotation=45)

    for bar, param in zip(bars, params):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                f'{param}M', ha='center', va='bottom', fontweight='bold')

    # 4. Precision vs Recall Scatter
    ax4 = plt.subplot(3, 3, 4)
    scatter = ax4.scatter(precision_scores, recall_scores, s=150, c=colors, alpha=0.7)

    for i, model in enumerate(models):
        ax4.annotate(model, (precision_scores[i], recall_scores[i]),
                    xytext=(5, 5), textcoords='offset points', fontsize=9)

    ax4.set_xlabel('Precision')
    ax4.set_ylabel('Recall')
    ax4.set_title('Precision vs Recall Trade-off', fontweight='bold')
    ax4.grid(True, alpha=0.3)

    # Add diagonal lines for F1 contours
    x_line = np.linspace(0, 1, 100)
    for f1_line in [0.6, 0.7, 0.8, 0.9]:
        y_line = (f1_line * x_line) / (2 * x_line - f1_line)
        y_line = np.where(y_line > 0, y_line, np.nan)
        ax4.plot(x_line, y_line, '--', alpha=0.3, label=f'F1={f1_line}')

    # 5. Entity Performance Radar Chart
    ax5 = plt.subplot(3, 3, 5, projection='polar')

    angles = np.linspace(0, 2 * np.pi, len(entities), endpoint=False).tolist()
    angles += angles[:1]  # Complete the circle

    for model_name, model_info in models_data.items():
        values = []
        for entity in entities:
            if entity in model_info['entities']:
                values.append(model_info['entities'][entity]['f1'])
            else:
                values.append(0.0)
        values += values[:1]  # Complete the circle

        ax5.plot(angles, values, 'o-', linewidth=2, label=model_name, color=model_info['color'])
        ax5.fill(angles, values, alpha=0.25, color=model_info['color'])

    ax5.set_xticks(angles[:-1])
    ax5.set_xticklabels(entities)
    ax5.set_ylim(0, 1)
    ax5.set_title('Entity Performance Radar Chart', fontweight='bold', pad=20)
    ax5.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

    # 6. Error Analysis (TP, FP, FN)
    ax6 = plt.subplot(3, 3, 6)
    tp_values = [info['details']['tp'] for info in models_data.values()]
    fp_values = [info['details']['fp'] for info in models_data.values()]
    fn_values = [info['details']['fn'] for info in models_data.values()]

    x = np.arange(len(models))
    width = 0.25

    ax6.bar(x - width, tp_values, width, label='True Positives', color='green', alpha=0.7)
    ax6.bar(x, fp_values, width, label='False Positives', color='red', alpha=0.7)
    ax6.bar(x + width, fn_values, width, label='False Negatives', color='orange', alpha=0.7)

    ax6.set_xlabel('Models')
    ax6.set_ylabel('Count')
    ax6.set_title('Error Analysis (TP/FP/FN)', fontweight='bold')
    ax6.set_xticks(x)
    ax6.set_xticklabels(models, rotation=15)
    ax6.legend()

    # 7. F1 Score Distribution by Entity
    ax7 = plt.subplot(3, 3, 7)
    entity_f1_data = []
    entity_labels = []

    for entity in entities:
        for model_name, model_info in models_data.items():
            if entity in model_info['entities']:
                entity_f1_data.append(model_info['entities'][entity]['f1'])
                entity_labels.append(f"{entity}\n{model_name}")

    if entity_f1_data:
        ax7.boxplot([entity_f1_data], labels=['All Entities'])
        ax7.set_ylabel('F1-Score')
        ax7.set_title('F1-Score Distribution', fontweight='bold')
        ax7.grid(True, alpha=0.3)

    # 8. Model Efficiency (F1 per Parameter)
    ax8 = plt.subplot(3, 3, 8)
    efficiency_scores = []
    for info in models_data.values():
        param_count = 355 if '355M' in info['parameters'] else 110
        efficiency = info['overall']['f1'] / param_count * 1000  # F1 per million parameters * 1000
        efficiency_scores.append(efficiency)

    bars = ax8.bar(models, efficiency_scores, color=colors, alpha=0.7)
    ax8.set_ylabel('F1-Score per Million Parameters (×1000)')
    ax8.set_title('Model Efficiency', fontweight='bold')
    ax8.tick_params(axis='x', rotation=45)

    for bar, eff in zip(bars, efficiency_scores):
        ax8.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{eff:.2f}', ha='center', va='bottom', fontweight='bold')

    # 9. Summary Statistics Table
    ax9 = plt.subplot(3, 3, 9)
    ax9.axis('tight')
    ax9.axis('off')

    summary_data = []
    for model_name, model_info in models_data.items():
        summary_data.append([
            model_name,
            f"{model_info['overall']['f1']:.3f}",
            f"{model_info['overall']['precision']:.3f}",
            f"{model_info['overall']['recall']:.3f}",
            model_info['parameters']
        ])

    table = ax9.table(cellText=summary_data,
                     colLabels=['Model', 'F1', 'Precision', 'Recall', 'Parameters'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    ax9.set_title('Summary Statistics', fontweight='bold', pad=20)

    plt.tight_layout()
    plt.savefig('comprehensive_model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✅ Static visualizations saved as 'comprehensive_model_comparison.png'")

# -------------------------
# 5) Create Interactive Visualizations
# -------------------------
def create_interactive_visualizations(models_data, overall_df, entity_df):
    """Create interactive Plotly visualizations"""

    if not models_data:
        return

    # 1. Interactive Overall Performance Comparison
    fig1 = go.Figure()

    models = [info['name'] for info in models_data.values()]
    colors = [info['color'] for info in models_data.values()]

    # F1 Scores
    fig1.add_trace(go.Bar(
        name='F1-Score',
        x=models,
        y=[info['overall']['f1'] for info in models_data.values()],
        marker_color=colors,
        text=[f"{info['overall']['f1']:.3f}" for info in models_data.values()],
        textposition='auto',
    ))

    fig1.update_layout(
        title='Interactive Overall Performance Comparison',
        xaxis_title='Models',
        yaxis_title='F1-Score',
        showlegend=False,
        template='plotly_white'
    )

    fig1.show()

    # 2. Interactive Entity-Level Heatmap
    entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]
    z_data = []
    y_labels = []

    for model_name, model_info in models_data.items():
        row = []
        for entity in entities:
            if entity in model_info['entities']:
                row.append(model_info['entities'][entity]['f1'])
            else:
                row.append(0.0)
        z_data.append(row)
        y_labels.append(model_name)

    fig2 = go.Figure(data=go.Heatmap(
        z=z_data,
        x=entities,
        y=y_labels,
        colorscale='RdYlGn',
        zmin=0,
        zmax=1,
        text=[[f'{val:.3f}' for val in row] for row in z_data],
        texttemplate="%{text}",
        textfont={"size": 12},
        hoverongaps=False
    ))

    fig2.update_layout(
        title='Interactive Entity-Level F1 Scores Heatmap',
        xaxis_title='Entity Types',
        yaxis_title='Models',
        template='plotly_white'
    )

    fig2.show()

    # 3. Interactive 3D Performance Visualization
    fig3 = go.Figure(data=go.Scatter3d(
        x=[info['overall']['precision'] for info in models_data.values()],
        y=[info['overall']['recall'] for info in models_data.values()],
        z=[info['overall']['f1'] for info in models_data.values()],
        mode='markers+text',
        marker=dict(
            size=12,
            color=[info['color'] for info in models_data.values()],
        ),
        text=models,
        textposition="top center"
    ))

    fig3.update_layout(
        title='3D Performance Space (Precision, Recall, F1)',
        scene=dict(
            xaxis_title='Precision',
            yaxis_title='Recall',
            zaxis_title='F1-Score'
        ),
        template='plotly_white'
    )

    fig3.show()

    print("✅ Interactive visualizations displayed")

# -------------------------
# 6) Generate Detailed Report
# -------------------------
def generate_detailed_report(models_data, overall_df, entity_df):
    """Generate comprehensive text report"""

    print("\n" + "="*80)
    print("📋 COMPREHENSIVE PHI DETECTION MODEL COMPARISON REPORT")
    print("="*80)

    if not models_data:
        print("❌ No model data available for analysis")
        return

    print(f"\n🔬 MODELS ANALYZED: {len(models_data)}")
    for model_name, model_info in models_data.items():
        print(f"   • {model_info['name']}: {model_info['type']} ({model_info['parameters']})")

    print(f"\n📊 OVERALL PERFORMANCE SUMMARY:")
    print(overall_df.round(4).to_string(index=False))

    # Best performing model
    best_f1_idx = overall_df['F1-Score'].idxmax()
    best_model = overall_df.iloc[best_f1_idx]

    print(f"\n🏆 BEST OVERALL PERFORMER:")
    print(f"   Model: {best_model['Model']}")
    print(f"   F1-Score: {best_model['F1-Score']:.4f}")
    print(f"   Precision: {best_model['Precision']:.4f}")
    print(f"   Recall: {best_model['Recall']:.4f}")
    print(f"   Parameters: {best_model['Parameters']}")

    # Entity-level analysis
    print(f"\n🎯 ENTITY-LEVEL PERFORMANCE ANALYSIS:")
    entities = ["DATE", "HOSPITAL", "ID", "LOCATION", "NAME", "PHONE"]

    for entity in entities:
        print(f"\n   {entity} Entity:")
        entity_scores = []
        for model_name, model_info in models_data.items():
            if entity in model_info['entities']:
                f1 = model_info['entities'][entity]['f1']
                entity_scores.append((model_name, f1))

        entity_scores.sort(key=lambda x: x[1], reverse=True)
        for i, (model, f1) in enumerate(entity_scores, 1):
            print(f"      {i}. {model:15s}: {f1:.3f}")

    # Model strengths and weaknesses
    print(f"\n💪 MODEL STRENGTHS AND WEAKNESSES:")
    for model_name, model_info in models_data.items():
        print(f"\n   {model_name}:")
        print(f"      Domain: {model_info['domain']}")
        print(f"      Overall F1: {model_info['overall']['f1']:.4f}")

        # Find best and worst entities
        entity_performance = []
        for entity in entities:
            if entity in model_info['entities']:
                entity_performance.append((entity, model_info['entities'][entity]['f1']))

        entity_performance.sort(key=lambda x: x[1], reverse=True)

        if entity_performance:
            best_entities = [e for e, f1 in entity_performance if f1 >= 0.7]
            weak_entities = [e for e, f1 in entity_performance if f1 < 0.5]

            if best_entities:
                print(f"      Strengths: {', '.join(best_entities)}")
            if weak_entities:
                print(f"      Weaknesses: {', '.join(weak_entities)}")

    # Recommendations
    print(f"\n🎯 RECOMMENDATIONS:")

    if len(models_data) >= 2:
        # Find most efficient model
        efficiency_scores = []
        for model_name, model_info in models_data.items():
            param_count = 355 if '355M' in model_info['parameters'] else 110
            efficiency = model_info['overall']['f1'] / param_count
            efficiency_scores.append((model_name, efficiency, model_info['overall']['f1']))

        efficiency_scores.sort(key=lambda x: x[1], reverse=True)
        most_efficient = efficiency_scores[0]

        print(f"   • Most Efficient: {most_efficient[0]} (F1: {most_efficient[2]:.4f})")
        print(f"   • Best Overall: {best_model['Model']} (F1: {best_model['F1-Score']:.4f})")

        if best_model['Parameters'] == '355M':
            print(f"   • For resource-constrained environments, consider smaller models")

        print(f"   • For production deployment, consider ensemble of top 2 models")

    print(f"\n✅ Report generation completed!")

# -------------------------
# 7) Main Execution Function
# -------------------------
def main():
    """Main execution function"""

    # Load all model results
    models_data = load_all_model_results()

    if not models_data:
        print("❌ No model results found. Please ensure evaluation files exist:")
        print("   • evaluation_results.json (ClinicalBERT)")
        print("   • biobert_evaluation_results.json (BioBERT)")
        print("   • roberta_large_evaluation_results.json (RoBERTa-Large)")
        return

    # Create comparison DataFrames
    overall_df, entity_df = create_comparison_dataframes(models_data)

    # Perform statistical analysis
    perform_statistical_analysis(models_data)

    # Create visualizations
    print(f"\n🎨 Creating comprehensive visualizations...")
    create_static_visualizations(models_data, overall_df, entity_df)

    print(f"\n🎨 Creating interactive visualizations...")
    create_interactive_visualizations(models_data, overall_df, entity_df)

    # Generate detailed report
    generate_detailed_report(models_data, overall_df, entity_df)

    # Save comparison data
    if overall_df is not None:
        overall_df.to_csv('model_comparison_overall.csv', index=False)
        print(f"✅ Overall comparison saved to 'model_comparison_overall.csv'")

    if entity_df is not None:
        entity_df.to_csv('model_comparison_entities.csv', index=False)
        print(f"✅ Entity comparison saved to 'model_comparison_entities.csv'")

    print(f"\n🎉 Comprehensive model comparison completed!")
    print(f"📁 Files generated:")
    print(f"   • comprehensive_model_comparison.png")
    print(f"   • model_comparison_overall.csv")
    print(f"   • model_comparison_entities.csv")

# Execute the comparison
if __name__ == "__main__":
    main()